# Pre-Processing für die Datenanalyse von Konstanz, Kreuzlingen, Landkreis Konstanz und dem Kanton Thurgau

Dieses Notebook ist nach thematischen Abschnitten gegliedert, um die Datenanalyse übersichtlich und reproduzierbar zu gestalten.

**Abschnittsübersicht:**
1. Import und Setup
2. Konstanz: Daten einlesen und bereinigen
3. Konstanz: Daten zusammenführen und exportieren
4. Konstanz: Wahldaten einlesen, bereinigen und analysieren
5. Konstanz: Demographie und Wahlergebnisse zusammenführen
6. Kreuzlingen: Daten einlesen, bereinigen und aggregieren
7. Kreuzlingen: Demographie und Wahlergebnisse zusammenführen
8. Landkreis Konstanz: Daten einlesen, bereinigen und aggregieren
9. Landkreis Konstanz: Demographie und Wahlergebnisse zusammenführen
10. Kanton Thurgau: Daten einlesen, bereinigen und aggregieren
11. Kanton Thurgau: Demographie und Wahlergebnisse zusammenführen

# 1. Import und Setup

In diesem Abschnitt werden alle notwendigen Bibliotheken für die Datenanalyse und Visualisierung importiert. Dazu gehören:

- **pandas**: Für Datenmanipulation und -analyse, insbesondere das Arbeiten mit DataFrames.
- **numpy**: Für numerische Operationen und effiziente Array-Verarbeitung.
- **seaborn** und **matplotlib.pyplot**: Für die Erstellung von Visualisierungen und Diagrammen.
- **os**: Für Dateipfad- und Verzeichnisoperationen.
- **re**: Für reguläre Ausdrücke, z.B. zur Spaltennamen- oder Dateinamensbereinigung.
- **datetime**: Für Datumsoperationen.
- **statsmodels.api**: Für statistische Analysen und Modellierung.

Diese Bibliotheken bilden die Grundlage für alle weiteren Datenverarbeitungs- und Analyseaufgaben im Notebook.

In [61]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import re
import datetime as dt
import statsmodels.api as sm

# 2. Konstanz: Daten einlesen und bereinigen

In diesem Abschnitt werden verschiedene Datensätze für die Stadt Konstanz eingelesen und bereinigt. Die Daten stammen vom Offenen Datenportal Konstanz [https://offenedaten-konstanz.de/search].

- **Einlesen der Rohdaten**: Die Daten stammen aus unterschiedlichen CSV-Dateien und enthalten Informationen zu Einwohnerzahlen, Arbeitsmarkt, Religion, Haushalten und Kaufkraft auf Stadtteilebene.
- **Bereinigung der Spalten und Werte**: Dazu gehören das Entfernen unerwünschter Zeichen (z.B. Punkte, Anführungszeichen), das Konvertieren von Datumsangaben in einheitliche Formate, das Umwandeln von Strings in numerische Werte und das Behandeln fehlender Werte.
- **Spezifische Bereinigung pro Datensatz**: Für jeden Datensatz werden individuelle Bereinigungsschritte durchgeführt, z.B. das Gruppieren nach Jahr, das Entfernen von Duplikaten oder das Anpassen von Spaltennamen.
- **Vorbereitung für spätere Zusammenführung**: Durch die Vereinheitlichung der Spalten und Formate wird sichergestellt, dass die verschiedenen Datensätze später problemlos zusammengeführt werden können.



In [ ]:
# Einwohnerdaten Stadtteile
df_kn_stadtviertel = pd.read_csv('data/Konstanz/Einwohner_nach_Stadtviertel_2010-2024_1.csv', sep=';', dtype=str)
# Arbeitsmarkt
df_kn_arbeitsmarkt = pd.read_csv('data/Konstanz/Arbeitsmarkt_Stadtteile_2011-2023_0_0.csv', sep=';')
# Religion
df_kn_religion = pd.read_csv('data/Konstanz/Einwohner_Religionszugehoerigkeit_Stadtteile_2010-2024_1.csv', sep=';')
# Haushalte
df_kn_haushalte = pd.read_csv('data/Konstanz/Haushalte_nach_Haushaltsgroeße_Stadtteile_2010-2024.csv', sep=';')
# Kaufkraft
df_kn_kaufkraft = pd.read_csv('data/Konstanz/Kaufkraftindex_Stadtteile_2011-2023_0.csv', sep=';')



In [63]:
# Einwohnerdaten: Datentypen und Spalten bereinigen
df_kn_stadtviertel['stt'] = df_kn_stadtviertel['stt'].astype(str)
df_kn_stadtviertel['stichtag'] = pd.to_datetime(df_kn_stadtviertel['stichtag'], format='%d.%m.%Y')
df_kn_stadtviertel['jahr'] = df_kn_stadtviertel['stichtag'].dt.year
df_kn_stadtviertel = df_kn_stadtviertel.drop(columns=['stichtag'])
df_kn_stadtviertel['stt'] = (
    df_kn_stadtviertel['stt']
    .str.replace('.', '0', regex=False)
    .str.replace('"', '', regex=False)
    .astype(int)
)
df_kn_stadtviertel['stadtviertel'] = (
    df_kn_stadtviertel['stadtviertel']
    .astype(str)
    .str.replace('.', '0', regex=False)
    .str.replace('"', '', regex=False)
    .astype(int)
)
cols_to_fix = [
    'einw_hw_gesamt', 'einw_hw_frauen', 'einw_hw_deutsch', 'einw_hw_nichtdeutsch',
    'einw_hw_unter18', 'einw_hw_18_bis_unter_30', 'einw_hw_30_bis_unter_40',
    'einw_hw_40_bis_unter_50', 'einw_hw_50_bis_unter_60', 'einw_hw_60_und_aelter',
    'haushalte'
]
for col in cols_to_fix:
    if col in df_kn_stadtviertel.columns:
        df_kn_stadtviertel[col] = (
            df_kn_stadtviertel[col]
            .str.replace('.', '', regex=False)
            .replace('nan', pd.NA)
            .astype('Int64')
        )

In [64]:
# Arbeitsmarkt: Datentypen und Spalten bereinigen
def convert_numeric_columns(df, date_col='stichtag', year_col='jahr', keep_cols=['stadtteil'], drop_cols=None):
    if drop_cols is not None:
        df = df.drop(columns=[col for col in drop_cols if col in df.columns])
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], format='%d.%m.%Y', errors='coerce')
        df[year_col] = df[date_col].dt.year
        df = df.drop(columns=[date_col])
    for col in ['stadtteil', 'Stadtteil']:
        if col in df.columns:
            df[col] = df[col].replace('Stadt Konstanz gesamt', 'Stadt Konstanz')
    for col in df.columns:
        if col in keep_cols:
            continue
        if df[col].dtype == object:
            df[col] = (
                df[col]
                .astype(str)
                .str.replace('%', '', regex=False)
                .str.replace(',', '.', regex=False)
                .str.replace(' ', '', regex=False)
                .str.replace('.0', '', regex=False)
            )
        df[col] = pd.to_numeric(df[col], errors='coerce') if col != year_col else df[col]
    return df

df_kn_arbeitsmarkt = convert_numeric_columns(
    df_kn_arbeitsmarkt,
    date_col='stichtag',
    year_col='jahr',
    keep_cols=['stadtteil'],
    drop_cols=['quelle', 'unnamed_7', 'ags']
)
df_kn_arbeitsmarkt['jahr'] = (
    df_kn_arbeitsmarkt['jahr']
    .astype(str)
    .str.replace('.0', '', regex=False)
    .replace('<NA>', pd.NA)
    .replace('nan', pd.NA)
    .astype('Int64')
)
if 'stadtteil_nr' in df_kn_arbeitsmarkt.columns:
    df_kn_arbeitsmarkt['stadtteil_nr'] = (
        df_kn_arbeitsmarkt['stadtteil_nr']
        .astype(str)
        .str.replace('.0', '', regex=False)
        .replace('<NA>', pd.NA)
        .replace('nan', pd.NA)
        .astype('Int64')
    )

In [65]:
# Haushalte: Datentypen und Spalten bereinigen

cols_to_fix = ['HH_3+-Personen'
]

for col in cols_to_fix:
    if col in df_kn_haushalte.columns:
        df_kn_haushalte[col] = (
            df_kn_haushalte[col]
            .astype(str)
            .str.replace('.0', '', regex=False)
            .str.replace('.', '', regex=False)
            .replace('nan', pd.NA)
            .astype('Int64')
        )

df_kn_haushalte = convert_numeric_columns(
    df_kn_haushalte,
    date_col='Stichtag',
    year_col='jahr',
    keep_cols=['Stadtteil'],
    drop_cols=['Quelle', 'AGS']
)
if 'HH_3+-Personen' in df_kn_haushalte.columns:
    df_kn_haushalte['HH_3+-Personen'] = (
        df_kn_haushalte['HH_3+-Personen']
        .astype(str)
        .str.replace('.0', '', regex=False)
        .replace('NaN', pd.NA)
        .astype('Int64')
    )

In [66]:
# Religion: Datentypen und Spalten bereinigen
df_kn_religion = convert_numeric_columns(
    df_kn_religion,
    date_col='stichtag',
    year_col='jahr',
    keep_cols=['stadtteil'],
    drop_cols=['quelle','ags']
)

In [67]:
# Kaufkraft: Datentypen und Spalten bereinigen
if 'kaufkraft_index_pro_einwohner' in df_kn_kaufkraft.columns:
    df_kn_kaufkraft['kaufkraft_index_pro_einwohner'] = (
        df_kn_kaufkraft['kaufkraft_index_pro_einwohner']
        .astype(str)
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
        .astype(float)
        .apply(np.ceil)
        .astype('Int64')
    )
df_kn_kaufkraft = convert_numeric_columns(
    df_kn_kaufkraft,
    date_col='stichtag',
    year_col='jahr',
    keep_cols=['stadtteil'],
    drop_cols=['quelle', 'ags']
)
if 'kaufkraft_index_pro_einwohner' in df_kn_kaufkraft.columns:
    df_kn_kaufkraft['kaufkraft_index_pro_einwohner'] = df_kn_kaufkraft['kaufkraft_index_pro_einwohner'].round(0).astype('Int64')


In [68]:
df_kn_kaufkraft[df_kn_kaufkraft['stadtteil']== 'Industriegebiet']

stadtteil_nr        stadtteil  jahr  kaufkraft_index_pro_einwohner
10             90  Industriegebiet  2023                             82
26             90  Industriegebiet  2022                             87
42             90  Industriegebiet  2021                             88
58             90  Industriegebiet  2020                             92
74             90  Industriegebiet  2019                             82
90             90  Industriegebiet  2018                             78
106            90  Industriegebiet  2017                             83
122            90  Industriegebiet  2016                             85
138            90  Industriegebiet  2015                             85
154            90  Industriegebiet  2014                             87
170            90  Industriegebiet  2013                             78
186            90  Industriegebiet  2012                             94
202            90  Industriegebiet  2011                             94

# 3. Konstanz: Daten zusammenführen und exportieren
Hier werden die zuvor bereinigten Datensätze pro Jahr auf Stadtteilebene zusammengeführt.

- **Filtern und Gruppieren nach Jahr**: Die Einwohnerdaten werden nach Jahr und Stadtteil gefiltert und gruppiert, um eine eindeutige Zuordnung zu ermöglichen.
- **Mergen der Datensätze**: Die verschiedenen Datenquellen (Einwohner, Arbeitsmarkt, Haushalte, Religion) werden über gemeinsame Schlüssel (z.B. Stadtteilnamen) zusammengeführt.
- **Bereinigung nach dem Merge**: Überflüssige oder doppelte Spalten werden entfernt, und relevante Spalten werden in numerische Typen konvertiert.
- **Export der kombinierten DataFrames**: Die zusammengeführten Daten werden als CSV-Dateien für jedes Jahr gespeichert.
- **Berechnung prozentualer Anteile und Quoten**: Es werden zusätzliche Kennzahlen wie Frauenanteil, Altersgruppenanteile, Arbeitslosenquote und durchschnittliche Haushaltsgröße berechnet und ebenfalls als CSV exportiert.

In [69]:
# Stadtviertel filtern und nach Jahr gruppieren
mask = df_kn_stadtviertel['stadtviertel'] == df_kn_stadtviertel['stt']
df_stadtviertel_filtered = df_kn_stadtviertel[mask].copy()
dfs_by_year = {}
for jahr in df_stadtviertel_filtered['jahr'].unique():
    df_jahr = df_stadtviertel_filtered[df_stadtviertel_filtered['jahr'] == jahr].copy()
    if df_jahr.duplicated(subset=['stt', 'stadtteil']).any():
        df_jahr = df_jahr.drop_duplicates(subset=['stt', 'stadtteil'])
    dfs_by_year[jahr] = df_jahr.reset_index(drop=True)

In [70]:
# alle Konstanzer Daten pro Jahr mergen
merged_by_year = {}
for jahr in dfs_by_year:
    df_base = dfs_by_year[jahr].copy()
    base_cols = [col for col in df_base.columns if col not in ['jahr.1', 'stadtteil.1', 'einw_hw_gesamt.1']]
    df_base = df_base[base_cols]
    df_merged = df_base.merge(
        df_kn_arbeitsmarkt[df_kn_arbeitsmarkt['jahr'] == jahr],
        on='stadtteil',
        how='left',
        suffixes=('', '_arbeitsmarkt')
    )
    df_merged = df_merged.merge(
        df_kn_haushalte[df_kn_haushalte['jahr'] == jahr],
        left_on='stadtteil', right_on='Stadtteil',
        how='left',
        suffixes=('', '_haushalte')
    )
    df_merged = df_merged.merge(
        df_kn_religion[df_kn_religion['jahr'] == jahr],
        on='stadtteil',
        how='left',
        suffixes=('', '_religion')
    )
    
    df_merged = df_merged.merge(
        df_kn_kaufkraft[df_kn_kaufkraft['jahr'] == jahr][['stadtteil', 'kaufkraft_index_pro_einwohner']],
        on='stadtteil',
        how='left'
    )
    # ---------------------------------------------------
    cols_to_remove = [col for col in df_merged.columns if col.endswith('.1') or col.endswith('_haushalte') or col.endswith('_arbeitsmarkt') or col.endswith('_religion')]
    for col in cols_to_remove:
        if col.replace('.1', '') in df_merged.columns:
            df_merged = df_merged.drop(columns=[col])
    cols_to_drop = [
        'Stadtteil', 'Stadtteil_Nr', 'wohnbevoelkerung insg.', 'stadtteil_nr', 'ags'
    ]
    for col in cols_to_drop:
        if col in df_merged.columns:
            df_merged = df_merged.drop(columns=col)
    cols_to_int = [
        'ags', 'stt', 'stadtviertel', 'einw_hw_gesamt', 'einw_hw_frauen', 'einw_hw_deutsch',
        'einw_hw_nichtdeutsch', 'einw_hw_unter18', 'einw_hw_18_bis_unter_30', 'einw_hw_30_bis_unter_40',
        'einw_hw_40_bis_unter_50', 'einw_hw_50_bis_unter_60', 'einw_hw_60_und_aelter','kaufkraft_index_pro_einwohner', 'haushalte',
        'jahr', 'stadtteil_nr', 'arbeitslose', 'sozialversicherungspflichtig_beschaeftigte',
        'HH_insgesamt', 'HH_1-Person', 'HH_2-Personen', 'HH_3+-Personen',
        'religion_katholisch', 'religion_evangelisch', 'religion_sonstige'
    ]
    for col in cols_to_int:
        if col in df_merged.columns:
            df_merged[col] = (
                df_merged[col]
                .astype(str)
                .str.replace('.0', '', regex=False)
                .replace(['nan', '<NA>'], pd.NA)
                .astype('Int64')
            )
    merged_by_year[jahr] = df_merged.reset_index(drop=True)

In [71]:
list(merged_by_year[2024].columns)

['gemeinde',
 'stt',
 'stadtteil',
 'stadtviertel',
 'einw_hw_gesamt',
 'einw_hw_frauen',
 'einw_hw_deutsch',
 'einw_hw_nichtdeutsch',
 'einw_hw_unter18',
 'einw_hw_18_bis_unter_30',
 'einw_hw_30_bis_unter_40',
 'einw_hw_40_bis_unter_50',
 'einw_hw_50_bis_unter_60',
 'einw_hw_60_und_aelter',
 'haushalte',
 'jahr',
 'arbeitslose',
 'sozialversicherungspflichtig_beschaeftigte',
 'HH_insgesamt',
 'HH_1-Person',
 'HH_2-Personen',
 'HH_3+-Personen',
 'religion_katholisch',
 'religion_evangelisch',
 'religion_sonstige',
 'kaufkraft_index_pro_einwohner']

In [72]:
# Exportiert die kombinierten DataFrames als CSV
output_dir = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing"
os.makedirs(output_dir, exist_ok=True)

for jahr, df in merged_by_year.items():
    df.to_csv(f"{output_dir}/df_konstanz_{jahr}.csv", index=False)

In [73]:
merged_by_year[2023]['kaufkraft_index_pro_einwohner']

0      98
1      96
2      80
3     113
4     101
5     121
6     118
7      87
8      90
9      82
10    129
11    138
12    135
13    133
14    140
15    100
Name: kaufkraft_index_pro_einwohner, dtype: Int64

In [74]:
# Altersdurchschnitt pro Stadtteil für 2024 berechnen (gewichtetes Mittel, da Einzelalter nicht vorhanden)

altersgruppen = [
    ('einw_hw_unter18', 9),
    ('einw_hw_18_bis_unter_30', 24),
    ('einw_hw_30_bis_unter_40', 35),
    ('einw_hw_40_bis_unter_50', 45),
    ('einw_hw_50_bis_unter_60', 55),
    ('einw_hw_60_und_aelter', 70)
]

def berechne_durchschnittsalter(row):
    gesamt = 0
    einwohner = 0
    for spalte, mittelwert in altersgruppen:
        n = row.get(spalte, 0)
        if pd.isna(n):
            n = 0
        gesamt += n * mittelwert
        einwohner += n
    return round(gesamt / einwohner, 1) if einwohner > 0 else None

df_avg_alter_24 = merged_by_year[2024].copy()
df_avg_alter_24['Durchschnittsalter'] = df_avg_alter_24.apply(berechne_durchschnittsalter, axis=1)

# Ausgabe: Stadtteil und Durchschnittsalter
df_avg_alter_kn = df_avg_alter_24[['stadtteil', 'Durchschnittsalter']]

df_avg_alter_kn.to_csv(f"{output_dir}/df_avg_alter_kn.csv", index=False)


In [75]:
df_avg_alter_kn

stadtteil  Durchschnittsalter
0            Altstadt                40.6
1            Paradies                40.9
2   Petershausen-West                38.8
3    Petershausen-Ost                45.6
4          Koenigsbau                37.5
5        Allmannsdorf                44.8
6               Staad                46.3
7        Fuerstenberg                41.4
8        Wollmatingen                41.0
9     Industriegebiet                38.0
10                Egg                40.3
11      Litzelstetten                46.6
12        Dingelsdorf                46.6
13          Dettingen                44.4
14         Wallhausen                45.5
15     Stadt Konstanz                41.6

In [76]:
df_avg_alter_kn[df_avg_alter_kn['stadtteil'] == 'Stadt Konstanz']

stadtteil  Durchschnittsalter
15  Stadt Konstanz                41.6

In [77]:
# Prozentuale Anteile und Quoten berechnen
def calc_anteile(df):
    result = pd.DataFrame()
    result['jahr'] = df['jahr']
    result['stadtteil'] = df['stadtteil']
    result['einw_hw_gesamt'] = df['einw_hw_gesamt']
    result['anteil_frauen'] = (df['einw_hw_frauen'] / df['einw_hw_gesamt'] * 100).round(2)
    result['anteil_maenner'] = ((1 - df['einw_hw_frauen'] / df['einw_hw_gesamt']) * 100).round(2)
    result['anteil_deutsch'] = (df['einw_hw_deutsch'] / df['einw_hw_gesamt'] * 100).round(2)
    result['anteil_nichtdeutsch'] = (df['einw_hw_nichtdeutsch'] / df['einw_hw_gesamt'] * 100).round(2)
    for col in [
        'einw_hw_unter18', 'einw_hw_18_bis_unter_30', 'einw_hw_30_bis_unter_40',
        'einw_hw_40_bis_unter_50', 'einw_hw_50_bis_unter_60', 'einw_hw_60_und_aelter'
    ]:
        result[f'anteil_{col}'] = (df[col] / df['einw_hw_gesamt'] * 100).round(2)
    if 'arbeitslose' in df.columns and 'sozialversicherungspflichtig_beschaeftigte' in df.columns:
        result['arbeitslosenquote'] = (
            df['arbeitslose'] / (df['arbeitslose'] + df['sozialversicherungspflichtig_beschaeftigte']) * 100
        ).round(2)
    result['durchschnitt_haushalt'] = (df['einw_hw_gesamt'] / df['haushalte']).round(2)
    for col in ['HH_1-Person', 'HH_2-Personen', 'HH_3+-Personen']:
        if col in df.columns and 'HH_insgesamt' in df.columns:
            result[f'anteil_{col}'] = (df[col] / df['HH_insgesamt'] * 100).round(2)
    return result

for jahr, df in merged_by_year.items():
    df_anteile = calc_anteile(df)
    df_anteile.to_csv(f"{output_dir}/df_konstanz_anteile_{jahr}.csv", index=False)

In [78]:
df_anteile_2024 = pd.read_csv('/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_konstanz_anteile_2024.csv')

In [79]:
df_anteile_2024

jahr          stadtteil  einw_hw_gesamt  anteil_frauen  anteil_maenner  \
0   2024           Altstadt           12392          52.22           47.78   
1   2024           Paradies            6004          53.01           46.99   
2   2024  Petershausen-West           16225          51.38           48.62   
3   2024   Petershausen-Ost            7353          55.03           44.97   
4   2024         Koenigsbau            6549          52.60           47.40   
5   2024       Allmannsdorf            5413          53.41           46.59   
6   2024              Staad            1819          53.60           46.40   
7   2024       Fuerstenberg           12459          51.99           48.01   
8   2024       Wollmatingen            6776          52.35           47.65   
9   2024    Industriegebiet            1214          39.46           60.54   
10  2024                Egg             802          50.75           49.25   
11  2024      Litzelstetten            3790          52.69           47.31   
12  2024        Dingelsdorf            2055          51.05           48.95   
13  2024          Dettingen            3389          52.29           47.71   
14  2024         Wallhausen            1128          51.95           48.05   
15  2024     Stadt Konstanz           87368          52.27           47.73   

    anteil_deutsch  anteil_nichtdeutsch  anteil_einw_hw_unter18  \
0            82.94                17.06                   13.59   
1            86.68                13.32                   10.44   
2            77.62                22.38                   13.84   
3            85.60                14.40                   13.07   
4            78.15                21.85                   16.49   
5            88.01                11.99                   12.73   
6            90.54                 9.46                   14.13   
7            79.17                20.83                   14.10   
8            82.36                17.64                   18.23   
9            47.69                52.31                   16.47   
10           86.53                13.47                   21.57   
11           88.58                11.42                   13.69   
12           90.95                 9.05                   14.40   
13           88.88                11.12                   17.85   
14           86.70                13.30                   14.72   
15           82.22                17.78                   14.30   

    anteil_einw_hw_18_bis_unter_30  anteil_einw_hw_30_bis_unter_40  \
0                            22.85                           16.44   
1                            31.91                           12.26   
2                            27.18                           17.28   
3                            17.35                           12.35   
4                            31.03                           11.59   
5                            19.62                           11.01   
6                            13.63                           11.32   
7                            22.99                           14.54   
8                            17.52                           13.03   
9                            25.86                           15.32   
10                           17.46                            7.48   
11                           14.62                           10.34   
12                           13.67                           10.46   
13                           12.19                           11.21   
14                           12.94                           12.77   
15                           22.52                           13.87   

    anteil_einw_hw_40_bis_unter_50  anteil_einw_hw_50_bis_unter_60  \
0                            12.38                           11.70   
1                             8.46                           11.19   
2                            10.51                           10.81   
3                             9.23                           11.85

In [80]:
# Lade die aktuellen Anteile für 2024
df_anteile_2024 = pd.read_csv(
    "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_konstanz_anteile_2024.csv"
)
# Lade die Arbeitslosenquote für 2023
df_anteile_2023 = pd.read_csv(
    "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_konstanz_anteile_2023.csv"
)
# Lade das Durchschnittsalter
df_avg_alter_kn = pd.read_csv(
    "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_avg_alter_kn.csv"
)

# Füge die Arbeitslosenquote aus 2023 hinzu (per Stadtteil)
arbeitslosenquote_2023 = df_anteile_2023.set_index("stadtteil")["arbeitslosenquote"]
df_karte = df_anteile_2024.copy()
df_karte["arbeitslosenquote"] = df_karte["stadtteil"].map(arbeitslosenquote_2023)

# Füge das Durchschnittsalter hinzu
durchschnittsalter = df_avg_alter_kn.set_index("stadtteil")["Durchschnittsalter"]
df_karte["Durchschnittsalter"] = df_karte["stadtteil"].map(durchschnittsalter)

# Optional: Spaltenreihenfolge anpassen
spalten = [
    "stadtteil",
    "einw_hw_gesamt",
    "Durchschnittsalter",
    "anteil_frauen",
    "anteil_maenner",
    "anteil_deutsch",
    "anteil_nichtdeutsch",
    "anteil_einw_hw_unter18",
    "anteil_einw_hw_18_bis_unter_30",
    "anteil_einw_hw_30_bis_unter_40",
    "anteil_einw_hw_40_bis_unter_50",
    "anteil_einw_hw_50_bis_unter_60",
    "anteil_einw_hw_60_und_aelter",
    "arbeitslosenquote",
    "durchschnitt_haushalt",
    "anteil_HH_1-Person",
    "anteil_HH_2-Personen",
    "anteil_HH_3+-Personen"
]
df_karte = df_karte[spalten]

# Export für die Karte
df_karte.to_csv("/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_karte_konstanz.csv", index=False)

## Die Daten zu den Bildungdabschlüssen sind aus dem Zensus 2022: https://www.zensus2022.de/DE/Aktuelles/Bildung_Erwerbstaetigkeit_VOE.html (aus der excel-Tablle übertragen)

In [81]:
# Personen im Alter von 15 Jahren und älter nach höchstem beruflichen Abschluss (ausführlich), Daten aus Zensus 2022 für Landkreis Konstanz 

import pandas as pd

daten = {
    "Abschluss": [
        "Lehre",
        "Fachschulabschluss",
        "Fachschulabschluss DDR",
        "Bachelor",
        "Master",
        "Diplom",
        "Promotion",
        "ohne beruflichen Abschluss",
        "davon noch in schulischer Ausbildung"
    ],
    "Anzahl": [
        84440,
        30650,
        1270,
        11720,
        7390,
        29560,
        5630,
        71720,
        10870
    ]
}

df_beruf_bildung_kn = pd.DataFrame(daten)
df_beruf_bildung_kn

Abschluss  Anzahl
0                                 Lehre   84440
1                    Fachschulabschluss   30650
2                Fachschulabschluss DDR    1270
3                              Bachelor   11720
4                                Master    7390
5                                Diplom   29560
6                             Promotion    5630
7            ohne beruflichen Abschluss   71720
8  davon noch in schulischer Ausbildung   10870

In [82]:
# Berufliche Bildungsabschlüsse: Hoch, Mittel, Niedrig (Konstanz)

hoch = 11720 + 7390 + 29560 + 5630  # Bachelor + Master + Diplom + Promotion
mittel = 84440 + 30650 + 1270       # Lehre + Fachschulabschluss + Fachschulabschluss DDR
niedrig = 71720 - 10870             # ohne beruflichen Abschluss abzüglich noch in Ausbildung

daten_kategorien = {
    "Kategorie": ["hoch", "mittel", "niedrig"],
    "Anzahl": [hoch, mittel, niedrig]
}

df_bildung_kategorien = pd.DataFrame(daten_kategorien)
df_bildung_kategorien






Kategorie  Anzahl
0      hoch   54300
1    mittel  116360
2   niedrig   60850

In [83]:
# Erwerbstätige Personen nach höchstem beruflichen Abschluss: hoch, mittel, niedrig, Landkreis Konstanz 

hoch = 39410  # Bachelor + Master + Diplom + Promotion
mittel = 51660 + 21690  # Lehre + Fachschulabschluss
niedrig = 33180  # ohne beruflichen Abschluss
gesamt = hoch + mittel + niedrig

df_beruf_kategorien = pd.DataFrame([{
    "gesamt": gesamt,
    "hoch": hoch,
    "mittel": mittel,
    "niedrig": niedrig
}])

df_beruf_kategorien["anteil_niedrig"] = (df_beruf_kategorien["niedrig"] / df_beruf_kategorien["gesamt"] * 100).round(2)
df_beruf_kategorien["anteil_mittel"] = (df_beruf_kategorien["mittel"] / df_beruf_kategorien["gesamt"] * 100).round(2)
df_beruf_kategorien["anteil_hoch"] = (df_beruf_kategorien["hoch"] / df_beruf_kategorien["gesamt"] * 100).round(2)
df_beruf_kategorien.to_csv("/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_bildung_kn.csv", index=False)
df_beruf_kategorien


gesamt   hoch  mittel  niedrig  anteil_niedrig  anteil_mittel  anteil_hoch
0  145940  39410   73350    33180           22.74          50.26         27.0

In [84]:
daten_schule = {
    "Abschluss": [
        "Gesamt",
        "davon noch in schulischer Ausbildung",
        "Haupt/Volksschulabschluss",
        "Polytechnische Oberschule",
        "Realschule/Mittlere Reife",
        "Fachhochschulreife/Abitur",
        "ohne allgemeinbildenden Schulabschluss"
    ],
    "Anzahl": [
        242390,
        10870,
        57210,
        3750,
        56300,
        97050,
        17220
    ]
}

df_schulabschluesse_kn = pd.DataFrame(daten_schule)
df_schulabschluesse_kn


Abschluss  Anzahl
0                                  Gesamt  242390
1    davon noch in schulischer Ausbildung   10870
2               Haupt/Volksschulabschluss   57210
3               Polytechnische Oberschule    3750
4               Realschule/Mittlere Reife   56300
5               Fachhochschulreife/Abitur   97050
6  ohne allgemeinbildenden Schulabschluss   17220

In [85]:
# Schulabschlüsse in die Kategorien hoch, mittel, niedrig einteilen

hoch = 97050  # Fachhochschulreife/Abitur
mittel = 56300 + 3750  # Realschule/Mittlere Reife + Polytechnische Oberschule
niedrig = 57210 + 17220 - 10870  # Haupt/Volksschulabschluss + ohne allgemeinbildenden Schulabschluss -  noch in schulischer Ausbildung

daten_schul_kategorien = {
    "Kategorie": ["hoch", "mittel", "niedrig"],
    "Anzahl": [hoch, mittel, niedrig]
}

df_schul_kategorien = pd.DataFrame(daten_schul_kategorien)
df_schul_kategorien

Kategorie  Anzahl
0      hoch   97050
1    mittel   60050
2   niedrig   63560

# 4. Konstanz: Wahldaten einlesen, bereinigen und analysieren


In diesem Abschnitt werden die Bundestagswahldaten für die Stadtteile Konstanz eingelesen und bereinigt. Die Daten zur Wahl stammen ebenfalls vom Offenen Datenportal Konstanz. 

- **Einlesen der Wahldaten**: Die Daten werden aus einer CSV-Datei geladen und enthalten die Stimmenanteile verschiedener Parteien pro Stadtteil und Jahr.
- **Bereinigung der Spaltennamen und Werte**: Die Spaltennamen werden vereinheitlicht (z.B. Umbenennung nach Stadtteilnamen), und Werte werden von Komma- in Punktnotation umgewandelt, um eine numerische Verarbeitung zu ermöglichen.
- **Konvertierung in numerische Formate**: Stimmenanteile werden als Fließkommazahlen gespeichert, um spätere Berechnungen und Visualisierungen zu erleichtern.
- **Export der bereinigten Wahldaten**: Die bereinigten Daten werden als CSV-Datei gespeichert und stehen für weitere Analysen zur Verfügung.

In [86]:
df_kn_bundestagswahlen = pd.read_csv('/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Konstanz/Bundestagswahl_vorl Ergebnis Stadtteile KN 2009 bis 2025.csv', sep=';')
stadtteil_mapping = {
    '000_stadt_konstanz': 'Stadt Konstanz',
    '010_altstadt': 'Altstadt',
    '020_paradies': 'Paradies',
    '030_petershausen_west': 'Petershausen-West',
    '035_petershausen_ost': 'Petershausen-Ost',
    '040_koenigsbau': 'Koenigsbau',
    '050_allmannsdorf': 'Allmannsdorf',
    '060_staad': 'Staad',
    '070_fuerstenberg': 'Fuerstenberg',
    '080_wollmatingen': 'Wollmatingen',
    '090_industriegebiet': 'Industriegebiet',
    '100_egg': 'Egg',
    '110_litzelstetten': 'Litzelstetten',
    '120_dingelsdorf': 'Dingelsdorf',
    '130_dettingen': 'Dettingen',
    '140_wallhausen': 'Wallhausen'
}
df_kn_bundestagswahlen = df_kn_bundestagswahlen.rename(columns=stadtteil_mapping)
stadtteile = [
    'Stadt Konstanz', 'Altstadt', 'Paradies', 'Petershausen-West', 'Petershausen-Ost',
    'Koenigsbau', 'Allmannsdorf', 'Staad', 'Fuerstenberg', 'Wollmatingen',
    'Industriegebiet', 'Egg', 'Litzelstetten', 'Dingelsdorf', 'Dettingen', 'Wallhausen'
]
for stadtteil in stadtteile:
    df_kn_bundestagswahlen[stadtteil] = df_kn_bundestagswahlen[stadtteil].astype(str).str.replace(',', '.', regex=False).astype(float)

In [87]:
# Export der bereinigten Wahldaten
df_kn_bundestagswahlen.to_csv(f"{output_dir}/df_kn_bundestagswahlen.csv", index=False)

# 5. Konstanz: Demographie und Wahlergebnisse zusammenführen

In diesem Abschnitt werden für jedes Wahljahr die jeweils aktuellsten verfügbaren Demographie-Daten mit den Wahlergebnissen pro Stadtteil zusammengeführt. 

- **Zusammenführen der Demographie-Daten**: Alle bereinigten Demographie-Daten werden in einem DataFrame zusammengefasst.
- **Umwandlung der Wahldaten ins Long- und dann ins Wide-Format**: Die Wahldaten werden so umstrukturiert, dass sie pro Stadtteil und Jahr vorliegen.
- **Hilfsfunktion für rückwirkende Auswahl**: Für jedes Wahljahr und jeden Stadtteil wird mithilfe einer Funktion das jeweils aktuellste verfügbare Demographie-Jahr ausgewählt (rückwirkend, falls für das Wahljahr selbst keine Daten vorliegen).
- **Kombination der Daten**: Die Demographie- und Wahldaten werden pro Stadtteil und Wahljahr kombiniert.
- **Export der kombinierten Daten**: Das Ergebnis wird als CSV-Datei gespeichert und steht für weitere Analysen zur Verfügung.

In [88]:
# Alle Demographie-Daten in einen DataFrame
merged_all_years = pd.concat(
    [merged_by_year[j].assign(demo_jahr=j) for j in merged_by_year],
    ignore_index=True
)
# Wahldaten in Long-Format
wahl_long = df_kn_bundestagswahlen.melt(
    id_vars=['jahr', 'art_des_ergebnisses'],
    value_vars=stadtteile,
    var_name='stadtteil',
    value_name='stimmenanteil'
)
wahl_pivot = wahl_long.pivot_table(
    index=['jahr', 'stadtteil'],
    columns='art_des_ergebnisses',
    values='stimmenanteil'
).reset_index()
# Hilfsfunktion: Finde für ein Wahljahr und Stadtteil das aktuellste Demographie-Jahr <= Wahljahr
def get_latest_demo_row(wahljahr, stadtteile, fields=None):
    """
    Gibt für ein Wahljahr und Stadtteil ein Dictionary mit den jeweils aktuellsten Werten für alle Felder zurück.
    Falls ein Wert fehlt, wird rückwärts das nächst verfügbare Jahr genommen.
    """
    df = merged_all_years[
        (merged_all_years['stadtteil'] == stadtteile) &
        (merged_all_years['jahr'] <= wahljahr)
    ].sort_values('jahr', ascending=False)
    if df.empty:
        return None
    # Wenn keine Felder angegeben, nimm alle Spalten außer 'jahr', 'stadtteil', 'demo_jahr'
    if fields is None:
        fields = [col for col in df.columns if col not in ['jahr', 'stadtteil', 'demo_jahr']]
    result = {}
    for field in fields:
        # Suche rückwärts nach erstem nicht-nan Wert für dieses Feld
        val = None
        for _, row in df.iterrows():
            if not pd.isna(row[field]):
                val = row[field]
                break
        result[field] = val
    # Jahr des zuletzt gefundenen Werts (für Doku)
    result['demo_jahr'] = df.iloc[0]['jahr']
    result['stadtteil'] = stadtteile
    return result

# Beispiel für die Anwendung:
rows = []
wahljahre = sorted(df_kn_bundestagswahlen['jahr'].unique())
fields = [col for col in merged_all_years.columns if col not in ['jahr', 'stadtteil', 'demo_jahr']]
for wahljahr in wahljahre:
    for stadtteil in stadtteile:
        demo_row = get_latest_demo_row(wahljahr, stadtteil, fields)
        if demo_row is not None:
            wahl_row = wahl_pivot[(wahl_pivot['jahr'] == wahljahr) & (wahl_pivot['stadtteil'] == stadtteil)]
            if not wahl_row.empty:
                merged = demo_row.copy()
                merged['wahljahr'] = wahljahr
                for col in wahl_pivot.columns:
                    if col not in ['jahr', 'stadtteil']:
                        merged[col] = wahl_row.iloc[0][col]
                rows.append(merged)
df_merged_all = pd.DataFrame(rows)
zaehlspalten = [
    'einw_hw_gesamt', 'einw_hw_frauen', 'einw_hw_deutsch', 'einw_hw_nichtdeutsch',
    'einw_hw_unter18', 'einw_hw_18_bis_unter_30', 'einw_hw_30_bis_unter_40',
    'einw_hw_40_bis_unter_50', 'einw_hw_50_bis_unter_60', 'einw_hw_60_und_aelter',
    'haushalte', 'HH_insgesamt', 'HH_1-Person', 'HH_2-Personen', 'HH_3+-Personen'
]
for col in zaehlspalten:
    if col in df_merged_all.columns:
        df_merged_all[col] = pd.to_numeric(df_merged_all[col], errors='coerce').astype('Int64')
cols = list(df_merged_all.columns)
if 'jahr' in cols:
    cols.remove('jahr')
ordered_cols = ['wahljahr', 'demo_jahr', 'stadtteil'] + [col for col in cols if col not in ['wahljahr', 'demo_jahr', 'stadtteil']]
df_merged_all = df_merged_all[ordered_cols]
df_merged_all.to_csv(f"{output_dir}/df_kn_wahlergebnis_demo.csv", index=False)

In [89]:
df_merged_all

wahljahr  demo_jahr          stadtteil  gemeinde  stt  stadtviertel  \
0       2013       2013     Stadt Konstanz  Konstanz    0             0   
1       2013       2013           Altstadt  Konstanz   10            10   
2       2013       2013           Paradies  Konstanz   20            20   
3       2013       2013  Petershausen-West  Konstanz   30            30   
4       2013       2013   Petershausen-Ost  Konstanz   35            35   
..       ...        ...                ...       ...  ...           ...   
59      2025       2024                Egg  Konstanz  100           100   
60      2025       2024      Litzelstetten  Konstanz  110           110   
61      2025       2024        Dingelsdorf  Konstanz  120           120   
62      2025       2024          Dettingen  Konstanz  130           130   
63      2025       2024         Wallhausen  Konstanz  140           140   

    einw_hw_gesamt  einw_hw_frauen  einw_hw_deutsch  einw_hw_nichtdeutsch  \
0            82557           43120            71903                 10654   
1            11923            6231            10347                  1576   
2             6030            3163             5198                   832   
3            14843            7597            12197                  2646   
4             6938            3851             6394                   544   
..             ...             ...              ...                   ...   
59             802             407              694                   108   
60            3790            1997             3357                   433   
61            2055            1049             1869                   186   
62            3389            1772             3012                   377   
63            1128             586              978                   150   

    ...   SPD  Sonstige  Tierschutzpartei  Volksabstimmung  Volt  \
0   ...  22.2       NaN               0.8              0.2   NaN   
1   ...  22.4       NaN               0.8              0.1   NaN   
2   ...  23.8       NaN               0.9              0.2   NaN   
3   ...  23.6       NaN               0.7              0.3   NaN   
4   ...  21.8       NaN               0.8              0.1   NaN   
..  ...   ...       ...               ...              ...   ...   
59  ...  14.0       NaN               0.2              NaN   1.6   
60  ...  14.6       NaN               0.8              NaN   0.8   
61  ...  13.4       NaN               1.1              NaN   0.4   
62  ...  16.3       NaN               0.7              NaN   0.8   
63  ...  12.7       NaN               1.3              NaN   1.3   

    Wahlbeteiligung  die Basis  pro Deutschland  sonstige  ÖDP  
0              74.6        NaN              0.0       NaN  0.4  
1              76.6        NaN              0.0       NaN  0.6  
2              74.5        NaN              0.0       NaN  0.2  
3              69.0        NaN              0.1       NaN  0.6  
4              83.0        NaN              0.0       NaN  0.3  
..              ...        ...              ...       ...  ...  
59              NaN        0.1              NaN       NaN  NaN  
60              NaN        0.3              NaN       NaN  NaN  
61              NaN        0.1              NaN       NaN  NaN  
62              NaN        0.1              NaN       NaN  NaN  
63              NaN        0.2              NaN       NaN  NaN  

[64 rows x 61 columns]

In [90]:
# Filter auf 2025 und bringe ins Long-Format
wahl_2025 = df_kn_bundestagswahlen[df_kn_bundestagswahlen['jahr'] == 2025]
stadtteile = [
    'Stadt Konstanz', 'Altstadt', 'Paradies', 'Petershausen-West', 'Petershausen-Ost',
    'Koenigsbau', 'Allmannsdorf', 'Staad', 'Fuerstenberg', 'Wollmatingen',
    'Industriegebiet', 'Egg', 'Litzelstetten', 'Dingelsdorf', 'Dettingen', 'Wallhausen'
]
wahl_long = wahl_2025.melt(
    id_vars=['jahr', 'art_des_ergebnisses'],
    value_vars=stadtteile,
    var_name='stadtteil',
    value_name='stimmenanteil'
)
wahl_pivot = wahl_long.pivot_table(
    index='stadtteil',
    columns='art_des_ergebnisses',
    values='stimmenanteil'
).reset_index()

# Merge mit df_karte
df_karte = pd.read_csv("/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_karte_konstanz.csv")
df_karte_merged = df_karte.merge(wahl_pivot, on="stadtteil", how="left")

# Export
df_karte_merged.to_csv("/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/output_preprocessing/df_karte_konstanz_wahl2025.csv", index=False)

In [91]:
df_karte_merged

stadtteil  einw_hw_gesamt  Durchschnittsalter  anteil_frauen  \
0            Altstadt           12392                40.6          52.22   
1            Paradies            6004                40.9          53.01   
2   Petershausen-West           16225                38.8          51.38   
3    Petershausen-Ost            7353                45.6          55.03   
4          Koenigsbau            6549                37.5          52.60   
5        Allmannsdorf            5413                44.8          53.41   
6               Staad            1819                46.3          53.60   
7        Fuerstenberg           12459                41.4          51.99   
8        Wollmatingen            6776                41.0          52.35   
9     Industriegebiet            1214                38.0          39.46   
10                Egg             802                40.3          50.75   
11      Litzelstetten            3790                46.6          52.69   
12        Dingelsdorf            2055                46.6          51.05   
13          Dettingen            3389                44.4          52.29   
14         Wallhausen            1128                45.5          51.95   
15     Stadt Konstanz           87368                41.6          52.27   

    anteil_maenner  anteil_deutsch  anteil_nichtdeutsch  \
0            47.78           82.94                17.06   
1            46.99           86.68                13.32   
2            48.62           77.62                22.38   
3            44.97           85.60                14.40   
4            47.40           78.15                21.85   
5            46.59           88.01                11.99   
6            46.40           90.54                 9.46   
7            48.01           79.17                20.83   
8            47.65           82.36                17.64   
9            60.54           47.69                52.31   
10           49.25           86.53                13.47   
11           47.31           88.58                11.42   
12           48.95           90.95                 9.05   
13           47.71           88.88                11.12   
14           48.05           86.70                13.30   
15           47.73           82.22                17.78   

    anteil_einw_hw_unter18  anteil_einw_hw_18_bis_unter_30  \
0                    13.59                           22.85   
1                    10.44                           31.91   
2                    13.84                           27.18   
3                    13.07                           17.35   
4                    16.49                           31.03   
5                    12.73                           19.62   
6                    14.13                           13.63   
7                    14.10                           22.99   
8                    18.23                           17.52   
9                    16.47                           25.86   
10                   21.57                           17.46   
11                   13.69                           14.62   
12                   14.40                           13.67   
13                   17.85                           12.19   
14                   14.72                           12.94   
15                   14.30                           22.52   

    anteil_einw_hw_30_bis_unter_40  ...  Die Partei  FDP  FREIE WAEHLER  \
0                            16.44  ...         0.4  5.8            0.6   
1                            12.26  ...         0.3  5.0            0.6   
2                            17.28  ...         0.5  5.6            0.8   
3                            12.35  ...         0.2  8.1            0.4   
4                            11.59  ...         0.6  6.0            0.6   
5                            11.01  ...         0.5  7.0            0.8   
6                            11.32  ...         0.5  6.3            0.9   
7                            14.54  ...         0.5  4.9    

# 6. Kreuzlingen: Daten einlesen, bereinigen und aggregieren

Hier werden die Datensätze für Kreuzlingen und den Bezirk Kreuzlingen eingelesen, bereinigt und aggregiert. Die Daten Stammen vom Datenportal des Kantons Thurgau: https://data.tg.ch/pages/start/ 

- **Einlesen der Rohdaten**: Die Daten werden aus CSV-Dateien geladen und enthalten Informationen zu Alter, Geschlecht, Nationalität und Konfession auf Gemeindeebene.
- **Bereinigung und Umwandlung der Werte**: Alters- und Bevölkerungsdaten werden in numerische Formate konvertiert, und Altersgruppen werden gebildet.
- **Berechnung von Altersdurchschnitten**: Der durchschnittliche Alter pro Gemeinde und Jahr wird berechnet, sowohl für den gesamten Bezirk als auch für das jeweils letzte Jahr.
- **Aggregation nach Jahr und Gemeinde**: Die Daten werden gruppiert und aggregiert, um eine übersichtliche Darstellung der Altersstruktur zu ermöglichen.
- **Export der berechneten Altersdurchschnitte**: Die Ergebnisse werden als CSV-Dateien gespeichert.

In [92]:
# Einlesen der Altersdaten für den Bezirk Kreuzlingen aus dem Kanton Thurgau.
pfad = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Kreuzlingen:Kanton Thurgau/"
df57 = pd.read_csv(pfad + "sk-stat-57.csv")

# Umwandlung der Spalten ALTER_CODE und ANZAHL_PERSONEN in numerische Werte.
# Dies ist notwendig, um mathematische Berechnungen (z.B. Mittelwert) durchführen zu können.
df57["ALTER"] = pd.to_numeric(df57["ALTER_CODE"], errors="coerce")
df57["ANZAHL_PERSONEN"] = pd.to_numeric(df57["ANZAHL_PERSONEN"], errors="coerce")

# Auswahl der Daten nur für den Bezirk Kreuzlingen.
# Dies ist relevant, falls die Datei mehrere Bezirke enthält.
df57_kr = df57[df57["BEZIRK_NAME"] == "Bezirk Kreuzlingen"]

# Berechnung des durchschnittlichen Alters für den gesamten Bezirk Kreuzlingen über alle Jahre hinweg.
# Dazu wird das gewichtete Mittel aller Altersgruppen berechnet.
gesamt_sum = (df57_kr["ALTER"] * df57_kr["ANZAHL_PERSONEN"]).sum()
gesamt_count = df57_kr["ANZAHL_PERSONEN"].sum()
altersdurchschnitt_kr = (gesamt_sum / gesamt_count).round(1)
print(f"Altersdurchschnitt Bezirk Kreuzlingen (gesamt): {altersdurchschnitt_kr:.2f}")

# Definition des letzten verfügbaren Jahres für die Auswertung.
letztes_jahr = 2024

# Filterung der Daten auf das letzte Jahr.
df57_kr_last = df57_kr[df57_kr["JAHR"] == letztes_jahr]

# Berechnung des durchschnittlichen Alters für den Bezirk Kreuzlingen im letzten Jahr.
gesamt_sum = (df57_kr_last["ALTER"] * df57_kr_last["ANZAHL_PERSONEN"]).sum()
gesamt_count = df57_kr_last["ANZAHL_PERSONEN"].sum()
altersdurchschnitt_kr = (gesamt_sum / gesamt_count).round(1)
print(f"Altersdurchschnitt Bezirk Kreuzlingen ({letztes_jahr}): {altersdurchschnitt_kr:.2f}")

# Berechnung des durchschnittlichen Alters pro Gemeinde und Jahr.
# Das gewichtete Mittel wird für jede Gemeinde und jedes Jahr separat berechnet.
df_avg = (
    df57_kr.groupby(["JAHR", "GEMEINDE_NAME"])
    .apply(lambda x: (x["ALTER"] * x["ANZAHL_PERSONEN"]).sum() / x["ANZAHL_PERSONEN"].sum())
    .reset_index(name="Altersdurchschnitt")
    .round(1)
)

# Auswahl der Ergebnisse für das letzte Jahr.
df_avg_alter_24 = df_avg[df_avg['JAHR'] == letztes_jahr]

# Export der berechneten Durchschnittsalter pro Gemeinde für das letzte Jahr als CSV-Datei.
df_avg_alter_24.to_csv(f"{output_dir}/df_avg_alter_kr.csv", index=False)
df_avg_alter_24



Altersdurchschnitt Bezirk Kreuzlingen (gesamt): 42.80
Altersdurchschnitt Bezirk Kreuzlingen (2024): 43.40


/var/folders/xb/3cn73kh15fx078zh0ryb_lrc0000gn/T/ipykernel_67769/2387098170.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x["ALTER"] * x["ANZAHL_PERSONEN"]).sum() / x["ANZAHL_PERSONEN"].sum())


JAHR   GEMEINDE_NAME  Altersdurchschnitt
126  2024          Altnau                43.4
127  2024     Bottighofen                45.8
128  2024      Ermatingen                46.1
129  2024      Gottlieben                45.3
130  2024       Güttingen                43.7
131  2024       Kemmental                41.1
132  2024     Kreuzlingen                43.2
133  2024  Langrickenbach                40.4
134  2024         Lengwil                41.4
135  2024   Münsterlingen                44.0
136  2024     Raperswilen                44.6
137  2024      Salenstein                47.9
138  2024      Tägerwilen                42.2
139  2024           Wäldi                42.6

In [93]:
# ==========================================
# Einlesen und Aggregation der Daten für den Bezirk Kreuzlingen
# Ziel: Pro Gemeinde demografische Merkmale strukturieren
# ==========================================

# Pfad zur Datenquelle für den Bezirk Kreuzlingen im Kanton Thurgau
pfad = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Kreuzlingen:Kanton Thurgau/"

# Einlesen der Altersstruktur-Daten (sk-stat-57)
df57 = pd.read_csv(pfad + "sk-stat-57.csv")

# Einlesen der Konfessionszugehörigkeit (sk-stat-62)
df62 = pd.read_csv(pfad + "sk-stat-62.csv")

# Einlesen der Staatsangehörigkeit und Geschlecht (sk-stat-59)
df59 = pd.read_csv(pfad + "sk-stat-59.csv")

# Filterung auf den Bezirk Kreuzlingen
# (in den Dateien können auch andere Bezirke enthalten sein)
df57 = df57[df57["BEZIRK_NAME"] == "Bezirk Kreuzlingen"]
df62 = df62[df62["BEZIRK_NAME"] == "Bezirk Kreuzlingen"]
df59 = df59[df59["BEZIRK_NAME"] == "Bezirk Kreuzlingen"]

# Definition von Altersgruppen
# Diese Gruppierung ermöglicht eine bessere Vergleichbarkeit
bins = [0, 17, 29, 39, 49, 59, 120]
labels = ["0-17", "18-29", "30-39", "40-49", "50-59", "60+"]

# Umwandlung der Alterscodes in numerische Werte
df57["ALTER"] = pd.to_numeric(df57["ALTER_CODE"], errors="coerce")

# Einteilung in Altersgruppen
df57["ALTERSGRUPPE"] = pd.cut(df57["ALTER"], bins=bins, labels=labels, right=True, include_lowest=True)

# Aggregation der Altersgruppen nach Jahr und Gemeinde
alter_agg = (
    df57.groupby(["JAHR", "GEMEINDE_NAME", "ALTERSGRUPPE"])["ANZAHL_PERSONEN"]
    .sum()
    .reset_index()
    .pivot(index=["JAHR", "GEMEINDE_NAME"], columns="ALTERSGRUPPE", values="ANZAHL_PERSONEN")
    .reset_index()
)

# Berechnung der Gesamtbevölkerung pro Gemeinde und Jahr
alter_agg["gesamtbevölkerung"] = alter_agg[labels].sum(axis=1)

# Aggregation der Konfessionen nach Jahr und Gemeinde
konf = (
    df62.groupby(["JAHR", "GEMEINDE_NAME", "KONFESSION_BEZEICHNUNG"])["ANZAHL_PERSONEN"]
    .sum()
    .reset_index()
    .pivot(index=["JAHR", "GEMEINDE_NAME"], columns="KONFESSION_BEZEICHNUNG", values="ANZAHL_PERSONEN")
    .reset_index()
)

# Aggregation der Staatsangehörigkeit und Geschlecht nach Jahr und Gemeinde
staatsb = (
    df59.groupby(["JAHR", "GEMEINDE_NAME", "GESCHLECHT_BEZEICHNUNG", "NATIONALITAET_BEZEICHNUNG"])["ANZAHL_PERSONEN"]
    .sum()
    .reset_index()
    .pivot(index=["JAHR", "GEMEINDE_NAME"], columns=["GESCHLECHT_BEZEICHNUNG", "NATIONALITAET_BEZEICHNUNG"], values="ANZAHL_PERSONEN")
    .reset_index()
)

# Anpassung der Spaltennamen aus dem Pivot-Index für bessere Lesbarkeit
staatsb.columns = [
    "_".join([str(i) for i in col if i]) if isinstance(col, tuple) else col
    for col in staatsb.columns.values
]

# Zusammenführen aller demografischen Daten zu einem Gesamt-Datensatz
# Merge über Jahr und Gemeinde
result = (
    alter_agg
    .merge(konf, on=["JAHR", "GEMEINDE_NAME"], how="left")
    .merge(staatsb, on=["JAHR", "GEMEINDE_NAME"], how="left")
)

# Ergebnis:
# Ein DataFrame „result“ mit allen demografischen Kennzahlen pro Gemeinde und Jahr
result

/var/folders/xb/3cn73kh15fx078zh0ryb_lrc0000gn/T/ipykernel_67769/3619754587.py:37: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df57.groupby(["JAHR", "GEMEINDE_NAME", "ALTERSGRUPPE"])["ANZAHL_PERSONEN"]


JAHR  GEMEINDE_NAME  0-17  18-29  30-39  40-49  50-59   60+  \
0    2015         Altnau   436    271    267    342    378   454   
1    2015    Bottighofen   353    217    249    334    380   633   
2    2015     Ermatingen   528    390    366    512    541   862   
3    2015     Gottlieben    53     24     51     54     62    76   
4    2015      Güttingen   295    236    175    215    277   376   
..    ...            ...   ...    ...    ...    ...    ...   ...   
135  2024  Münsterlingen   616    434    452    489    567  1073   
136  2024    Raperswilen    70     45     62     46     60   139   
137  2024     Salenstein   212    112    161    164    259   539   
138  2024     Tägerwilen  1035    604    776    758    903  1338   
139  2024          Wäldi   233    117    133    176    187   309   

     gesamtbevölkerung  Evangelisch-reformiert  Römisch-katholisch  Übrige  \
0                 2148                     948                 622     578   
1                 2166                     903                 634     629   
2                 3199                    1379                 903     917   
3                  320                     136                 106      78   
4                 1574                     654                 464     456   
..                 ...                     ...                 ...     ...   
135               3631                     961                 993    1677   
136                422                     195                  74     153   
137               1447                     475                 319     653   
138               5414                    1506                1289    2619   
139               1155                     413                 218     524   

     Männlich_Ausland  Männlich_Schweiz  Weiblich_Ausland  Weiblich_Schweiz  
0                 215               847               195               891  
1                 336               763               318               749  
2                 499              1085               430              1185  
3                  73                91                56               100  
4                 160               634               142               638  
..                ...               ...               ...               ...  
135               747              1060               682              1142  
136                33               179                21               189  
137               195               517               212               523  
138              1104              1634               993              1683  
139               131               470               103               451  

[140 rows x 16 columns]

In [94]:
# ==========================================
# Einlesen und Aufbereitung der Bildungsdaten 
# für den Kanton Thurgau (Stand 2024)
# ==========================================

# Dateipfad zur Excel-Datei mit der höchsten abgeschlossenen Ausbildung
# nach Gemeinden und Bezirken
file_path = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Kreuzlingen:Kanton Thurgau/Registrierte_Arbeitslose_nach_Gemeinden_2010_2024.xlsx"

# Einlesen der relevanten Spalten aus der Excel-Datei
# - skiprows=6: Überspringt Kopfzeilen
# - skipfooter=12: Entfernt Fußnoten am Ende
# - usecols: Auswahl nur der benötigten Spalten
df_arbeit_ktg = pd.read_excel(
    file_path,
    sheet_name="Verhältnis AL Bevölkerung VJ",
    skiprows=2,
    skipfooter=4,
    engine="openpyxl"
)

df_arbeit_ktg = df_arbeit_ktg.drop(columns=["BFS-Nr.1"])

# 2. umbenennen
df_arbeit_ktg = df_arbeit_ktg.rename(columns={"Unnamed: 1": "Gebiet"})

# 3. Jahre in lange Form
df_arbeit_ktg = df_arbeit_ktg.melt(
    id_vars="Gebiet",
    var_name="Jahr",
    value_name="Arbeitslosenquote"
)

# 4. falls Jahr numerisch soll:
df_arbeit_ktg["Jahr"] = df_arbeit_ktg["Jahr"].astype(int)
df_arbeit_ktg['Arbeitslosenquote'] = df_arbeit_ktg['Arbeitslosenquote'].round(1)

# Ergebnis

df_arbeit_ktg

Gebiet  Jahr  Arbeitslosenquote
0              Kanton Thurgau  2010                2.4
1                Bezirk Arbon  2010                3.0
2                    Amriswil  2010                3.3
3                       Arbon  2010                3.8
4                      Dozwil  2010                1.1
...                       ...   ...                ...
1285                  Sulgen   2024                1.6
1286               Weinfelden  2024                1.7
1287              Wigoltingen  2024                1.1
1288                 Wuppenau  2024                0.7
1289  Zihlschlacht-Sitterdorf  2024                0.8

[1290 rows x 3 columns]

In [95]:
df_arbeit_ktg["Gebiet"] = df_arbeit_ktg["Gebiet"].astype(str).str.strip()
result["GEMEINDE_NAME"] = result["GEMEINDE_NAME"].astype(str).str.strip()

# 3. merge
result = result.merge(
    df_arbeit_ktg,
    left_on=["JAHR", "GEMEINDE_NAME"],
    right_on=["Jahr", "Gebiet"],
    how="left"
)

# 4. aufräumen: redundant
result = result.drop(columns=["Jahr", "Gebiet"])

# Ergebnis anschauen
result.head()

JAHR GEMEINDE_NAME  0-17  18-29  30-39  40-49  50-59  60+  \
0  2015        Altnau   436    271    267    342    378  454   
1  2015   Bottighofen   353    217    249    334    380  633   
2  2015    Ermatingen   528    390    366    512    541  862   
3  2015    Gottlieben    53     24     51     54     62   76   
4  2015     Güttingen   295    236    175    215    277  376   

   gesamtbevölkerung  Evangelisch-reformiert  Römisch-katholisch  Übrige  \
0               2148                     948                 622     578   
1               2166                     903                 634     629   
2               3199                    1379                 903     917   
3                320                     136                 106      78   
4               1574                     654                 464     456   

   Männlich_Ausland  Männlich_Schweiz  Weiblich_Ausland  Weiblich_Schweiz  \
0               215               847               195               891   
1               336               763               318               749   
2               499              1085               430              1185   
3                73                91                56               100   
4               160               634               142               638   

   Arbeitslosenquote  
0                1.4  
1                1.9  
2                2.1  
3                1.1  
4                1.4

# 7. Kreuzlingen: Demographie und Wahlergebnisse zusammenführen

In diesem Abschnitt werden die demographischen Daten mit den Wahlergebnissen für Kreuzlingen zusammengeführt. 

- **Einlesen der Parteistärken**: Die Wahlergebnisse (Parteistärken) werden aus Excel-Dateien für verschiedene Jahre eingelesen.
- **Bereinigung und Vereinheitlichung der Spalten**: Die Spaltennamen werden je nach Jahr angepasst und die Werte in numerische Formate konvertiert.
- **Zusammenführen mit den Demographie-Daten**: Die Parteistärken werden mit den aggregierten demographischen Daten auf Gemeindeebene gemerged.
- **Export der kombinierten Daten**: Die zusammengeführten Daten werden als CSV-Datei gespeichert und stehen für weitere Analysen zur Verfügung.


In [96]:
# ==========================================
# Einlesen und Bereinigung der Parteistärken 
# für den Kanton Thurgau (Gemeinde-Ebene)
# ==========================================

def load_parteistaerken(file_path, jahr):
    """
    Lädt ein Excel-Dokument mit Parteistärken (Wahlanteilen) aus dem
    Kanton Thurgau auf Gemeindeebene, bereinigt und formatiert es
    je nach Jahrgang, da sich die Parteibezeichnungen geändert haben.

    Parameter
    ----------
    file_path : str
        Pfad zur Excel-Datei
    jahr : int
        Wahljahr (2015, 2019 oder 2023)

    Rückgabe
    ----------
    DataFrame
        Parteistärken je Gemeinde, numerisch bereinigt
    """

    # Einlesen des Excel-Files, Start ab Zeile 3 (header=2) und
    # Entfernen der letzten 4 Fussnotenzeilen (skipfooter)
    df = pd.read_excel(file_path, header=2, skipfooter=4, engine="openpyxl")

    # Entfernen der Zeilen mit Aggregaten oder doppelten Bezeichnungen
    # (z.B. die erste Gemeinde-Zeile wird manchmal wiederholt)
    df_clean = df.drop([0, 2]).reset_index(drop=True)

    # Spaltennamen je nach Wahljahr festlegen
    if jahr == 2015:
        df_clean.columns = [
            "Gebiet", "SVP", "CVP", "FDP", "SP", "glp", "GP", "BDP", "EDU", "EVP", "DPS"
        ]
    elif jahr == 2019:
        df_clean.columns = [
            "Gebiet", "SVP", "CVP", "FDP", "SP", "glp", "GP", "BDP", "EDU", "EVP"
        ]
    elif jahr == 2023:
        df_clean.columns = [
            "Gebiet", "SVP", "Die Mitte", "FDP", "SP", "GLP", "GRÜNE", "EDU", "EVP", "AUFTG", "Mass-Voll"
        ]

    # Konvertieren aller Parteiwert-Spalten (außer Gebiet) zu numerisch
    # und runden auf eine Nachkommastelle
    for col in df_clean.columns[1:]:
        df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce").round(1)

    return df_clean


# ==========================================
# Einlesen der Parteistärken-Dateien
# ==========================================

# Pfade zu den Parteistärken-Excels aus dem Kanton Thurgau
file_2015 = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Kreuzlingen:Kanton Thurgau/2015 Parteistaerken nach Gemeinden und Bezirken.xlsx"
file_2019 = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Kreuzlingen:Kanton Thurgau/2019_Parteistaerken nach Gemeinden und Bezirken.xlsx"
file_2023 = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Kreuzlingen:Kanton Thurgau/t2_2023_Parteistaerken_nach_Gemeinden.xlsx"

# Einlesen der Daten pro Jahr
kt_parteistaerke_2015 = load_parteistaerken(file_2015, 2015)
kt_parteistaerke_2019 = load_parteistaerken(file_2019, 2019)
kt_parteistaerke_2023 = load_parteistaerken(file_2023, 2023)

# Beispielhafte Ausgabe zur Kontrolle
kt_parteistaerke_2015.head()


Gebiet   SVP   CVP   FDP    SP  glp   GP  BDP   EDU  EVP  DPS
0  Kanton Thurgau  39.9  13.1  13.0  12.7  6.2  5.4  3.8   3.4  2.3  0.2
1    Bezirk Arbon  38.3  11.7  15.2  14.2  4.8  5.8  3.4   3.9  2.4  0.2
2        Amriswil  40.0  10.9  16.6  11.6  4.1  4.9  4.0   5.3  2.4  0.2
3           Arbon  33.6  16.0  14.3  18.4  4.2  6.4  2.5   3.0  1.6  0.1
4          Dozwil  51.8   5.5   8.2   6.0  2.5  3.8  3.2  12.9  6.1  0.0

In [112]:
# ==========================================
# Zusammenführen von Parteistärken und Demographie
# im Bezirk Kreuzlingen
# ==========================================

# Mapping der zuvor eingelesenen Parteistärken-DataFrames auf die jeweiligen Jahre
# Dies erlaubt es, später einfach per Schleife die passenden Daten zu verknüpfen
parteistaerke_dict = {
    2015: kt_parteistaerke_2015,
    2019: kt_parteistaerke_2019,
    2023: kt_parteistaerke_2023
}

# Leere Liste zur Aufnahme der gemergten Ergebnisse je Jahr
merged_results = []

# Iteration über die Jahre
for jahr, partei_df in parteistaerke_dict.items():
    # Filterung des zuvor erzeugten 'result'-DataFrames auf das aktuelle Jahr
    result_jahr = result[result["JAHR"] == jahr].copy()
    
    # Merge der demographischen Daten mit den Parteistärken
    # Zusammenführung über den Gemeindename („Gebiet“) sowie das Wahljahr
    merged = result_jahr.merge(
        partei_df,
        left_on="GEMEINDE_NAME",
        right_on="Gebiet",
        how="left",   # auch Gemeinden ohne Parteistärken bleiben enthalten
        suffixes=('', f'_parteistaerke_{jahr}')
    )
    
    # Speichern des Wahljahres als eigene Spalte
    merged["WAHLJAHR"] = jahr
    
    # Ergebnis für dieses Jahr zur Liste hinzufügen
    merged_results.append(merged)

# Alle Jahres-DataFrames zusammenführen
# Dadurch entsteht ein konsistenter Gesamt-Datensatz
df_result_merged = pd.concat(merged_results, ignore_index=True)



# Export des Endergebnisses für den Bezirk Kreuzlingen
# enthält alle demographischen Variablen + Parteistärken
df_result_merged.to_csv(f"{output_dir}/df_kreuzlingen_demographie_parteistaerke.csv", index=False)

# Beispielhafte Ausgabe der zusammengeführten Daten
df_result_merged.head()


JAHR GEMEINDE_NAME  0-17  18-29  30-39  40-49  50-59  60+  \
0  2015        Altnau   436    271    267    342    378  454   
1  2015   Bottighofen   353    217    249    334    380  633   
2  2015    Ermatingen   528    390    366    512    541  862   
3  2015    Gottlieben    53     24     51     54     62   76   
4  2015     Güttingen   295    236    175    215    277  376   

   gesamtbevölkerung  Evangelisch-reformiert  ...  BDP  EDU  EVP  DPS  \
0               2148                     948  ...  3.3  2.2  1.1  0.0   
1               2166                     903  ...  3.0  1.2  1.4  0.0   
2               3199                    1379  ...  4.6  0.9  1.4  0.3   
3                320                     136  ...  5.9  2.6  3.8  0.0   
4               1574                     654  ...  3.8  2.5  1.7  0.1   

   WAHLJAHR  Die Mitte  GLP GRÜNE  AUFTG  Mass-Voll  
0      2015        NaN  NaN   NaN    NaN        NaN  
1      2015        NaN  NaN   NaN    NaN        NaN  
2      2015        NaN  NaN   NaN    NaN        NaN  
3      2015        NaN  NaN   NaN    NaN        NaN  
4      2015        NaN  NaN   NaN    NaN        NaN  

[5 rows x 34 columns]

# 8. Landkreis Konstanz: Daten einlesen, bereinigen und aggregieren
In diesem Abschnitt werden die Bevölkerungsstruktur-Daten für den Landkreis Konstanz aus mehreren CSV-Dateien eingelesen, bereinigt und aggregiert. Die Daten kommen vom Statistischen Landesamt Baden-Württemberg: https://www.statistik-bw.de/BevoelkGebiet/ 

- **Einlesen der Bevölkerungsstruktur-Daten**: Die Daten werden aus mehreren CSV-Dateien geladen, die jeweils ein Jahr abdecken.
- **Bereinigung der Bevölkerungsstruktur-Daten**: Bevölkerungsstruktur-Daten werden vereinheitlicht und in numerische Werte umgewandelt.
- **Bildung von Altersgruppen**: Die Altersdaten werden in vordefinierte Gruppen (z.B. 0-17, 18-29, etc.) eingeteilt.
- **Aggregation nach Jahr und Altersgruppe**: Die Daten werden gruppiert und aggregiert, um die Gesamtbevölkerung und die Verteilung auf die Altersgruppen zu berechnen.
- **Berechnung prozentualer Anteile**: Für jede Bevölkerungsstruktur-Daten -Variable und Jahr werden die prozentualen Anteile berechnet.
- **Erstellung von Pivot-Tabellen**: Die Daten werden so umstrukturiert, dass Altersgruppen als Spalten und Jahre als Zeilen dargestellt werden.
- **Export der aggregierten und prozentualen Daten**: Die Ergebnisse werden als CSV-Dateien gespeichert.


In [98]:
# ==========================================
# Einlesen und Bereinigung der Altersdaten 
# für den Landkreis Konstanz
# ==========================================

# Verzeichnis, in dem die CSV-Dateien liegen
datadir = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Landkreis Konstanz/"

# Hilfsfunktion, um aus dem Dateinamen das Jahr zu extrahieren
# Die Dateinamen enthalten einen Schlüssel wie z.B. 01035220.csv,
# wobei die letzten beiden Ziffern für das Jahr stehen.
def extract_year(filename):
    """
    Extrahiert das Jahr aus dem Dateinamen, basierend auf den
    zweistelligen Jahrencodes laut amtlicher Benennung.
    """
    match = re.search(r'010352(\d+)\.csv', filename)
    if match:
        code = match.group(1)
        code_to_year = {
            "00": 2014, "10": 2020, "20": 2021, "40": 2023,
            "50": 2015, "60": 2016, "70": 2017, "80": 2018, "90": 2019
        }
        return code_to_year.get(code, None)
    return None

# Alle relevanten CSV-Dateien im Verzeichnis finden
files = [
    f for f in os.listdir(datadir)
    if f.startswith("010352") and f.endswith(".csv")
]

# Hilfsfunktion zur Bereinigung der Altersangaben
# Manche Altersklassen heißen z.B. „unter 1 Jahr“ oder „80 u.m.Jahren“.
# Diese werden in numerische Werte überführt.
def clean_alter_column(df):
    """
    Bereinigt die Spalte 'alter' und wandelt sie in eine numerische Spalte
    alter_clean um, damit sie für Analysen geeignet ist.
    """
    df['alter_clean'] = df['alter'].replace('unter 1 Jahr', '0')
    df['alter_clean'] = df['alter_clean'].str.replace(r'u\\.m\\.Jahren', '', regex=True)
    df['alter_clean'] = df['alter_clean'].str.replace(r'Jahren', '', regex=True)
    df['alter_clean'] = df['alter_clean'].str.replace(r'Jahr', '', regex=True)
    df['alter_clean'] = df['alter_clean'].str.strip()
    df['alter_clean'] = pd.to_numeric(df['alter_clean'], errors='coerce').astype('Int64')
    df = df.drop(columns=['alter'])
    return df

# Dictionary zur Ablage aller bereinigten DataFrames je Jahr
dfs = {}

# Verarbeitung aller Dateien im Verzeichnis
for file in sorted(files):
    year = extract_year(file)
    if year is None:
        continue  # falls kein Jahr erkannt, Datei überspringen

    # Einlesen der CSV-Datei
    df = pd.read_csv(
        os.path.join(datadir, file),
        sep=";",
        engine="python",
        dtype=str
    )
    
    # Entfernen der letzten 3 Summenzeilen
    df = df.iloc[:-3]
    
    # Spaltennamen trimmen
    df.columns = df.columns.str.strip()
    
    # Punktzeichen aus allen Zahlenwerten entfernen
    for col in df.columns:
        if col != 'alter':
            df[col] = df[col].str.replace('.', '', regex=False)
    
    # Alters-Spalte bereinigen
    if 'alter' in df.columns:
        df = clean_alter_column(df)
    
    # Speichern im Dictionary mit Jahreszahl als Schlüssel
    dfs[year] = df

# Ergebnis:
# Im Dictionary dfs liegen nun für jedes Jahr bereinigte
# Altersstrukturdaten für den Landkreis Konstanz vor.


In [99]:
dfs[2023]

insgesamt_dt männlich_dt weiblich_st zusammen_aus männlich_aus  \
0          2370        1221        1149          431          215   
1          2467        1243        1224          448          229   
2          2937        1541        1396          463          222   
3          2810        1495        1315          482          267   
4          2790        1438        1352          492          278   
..          ...         ...         ...          ...          ...   
86         1606         631         975           59           32   
87         1407         575         832           63           34   
88         1251         470         781           54           22   
89          971         354         617           43           24   
90         2960         947        2013          106           37   

   weiblich_aus  alter_clean  
0           216            0  
1           219            1  
2           241            2  
3           215            3  
4           214            4  
..          ...          ...  
86           27           86  
87           29           87  
88           32           88  
89           19           89  
90           69         <NA>  

[91 rows x 7 columns]

In [100]:
# ==========================================
# Bildung und Aggregation von Altersgruppen 
# für den Landkreis Konstanz
# ==========================================

# Definition der Altersgruppen (Bins)
bins = [0, 17, 29, 39, 49, 59, 120]
labels = ["0-17", "18-29", "30-39", "40-49", "50-59", "60+"]

# Hilfsfunktion, um Altersgruppen zu aggregieren
def group_alter(df, jahr):
    """
    Gruppiert die Altersstruktur-Daten in definierte Altersgruppen
    und summiert die Bevölkerungszahlen pro Gruppe.

    Parameter
    ----------
    df : DataFrame
        Altersdaten mit Spalte 'alter_clean'
    jahr : int
        Jahr, aus dem die Daten stammen

    Rückgabe
    ----------
    DataFrame
        Aggregierte Bevölkerung pro Altersgruppe
    """

    # Altersgruppen-Segmentierung
    df['alter_group'] = pd.cut(
        df['alter_clean'],
        bins=bins,
        labels=labels,
        right=True,
        include_lowest=True
    )

    # Auswahl aller numerischen Spalten außer Alters-Spalten
    num_cols = [col for col in df.columns if col not in ['alter', 'alter_clean', 'alter_group']]

    # Konvertiere Zahlen als numerisch
    df[num_cols] = df[num_cols].apply(pd.to_numeric, errors='coerce')

    # Aggregation nach Altersgruppe
    grouped = (
        df.groupby('alter_group')[num_cols]
        .sum(min_count=1)
        .reset_index()
    )

    # Jahr ergänzen
    grouped['jahr'] = jahr
    
    return grouped

# Leere Liste zur Aufnahme der Jahres-Daten
all_years = []

# Iteration über die Jahre im Dictionary
for jahr, df in dfs.items():
    grouped = group_alter(df, jahr)
    all_years.append(grouped)

# Zusammenführen aller Jahre
df_landkreis_kn = pd.concat(all_years, ignore_index=True)

# Spaltenreihenfolge anpassen: Jahr zuerst
cols = ['jahr'] + [col for col in df_landkreis_kn.columns if col != 'jahr']
df_landkreis_kn = df_landkreis_kn[cols]

# Sortierung nach Jahr und Altersgruppe
df_landkreis_kn = df_landkreis_kn.sort_values(by=['jahr', 'alter_group']).reset_index(drop=True)

# Berechnung der Gesamtbevölkerung pro Altersgruppe und Jahr
# Summe von Deutschen und ausländischen Bürgern
df_landkreis_kn['gesamtbevölkerung'] = (
    df_landkreis_kn['insgesamt_dt'] + df_landkreis_kn['zusammen_aus']
)

# Export des fertigen DataFrames
df_landkreis_kn.to_csv(f"{output_dir}/df_landkreis_kn.csv", index=False)


/var/folders/xb/3cn73kh15fx078zh0ryb_lrc0000gn/T/ipykernel_67769/1191138484.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('alter_group')[num_cols]
/var/folders/xb/3cn73kh15fx078zh0ryb_lrc0000gn/T/ipykernel_67769/1191138484.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('alter_group')[num_cols]
/var/folders/xb/3cn73kh15fx078zh0ryb_lrc0000gn/T/ipykernel_67769/1191138484.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future d

In [101]:
# DataFrame für 2023 aus dem Dictionary holen
df_2023 = dfs[2023]

# Gesamtbevölkerung pro Altersjahr
df_2023["gesamt"] = df_2023["insgesamt_dt"] + df_2023["zusammen_aus"]

# Alterswerte mit Gesamtbevölkerung gewichten
# Zeilen mit fehlendem alter_clean (z.B. <NA>) ausschließen
valid = df_2023[df_2023["alter_clean"].notna()]

gewichtete_summe = (valid["alter_clean"] * valid["gesamt"]).sum()
gesamtbevölkerung = valid["gesamt"].sum()

# Durchschnitt berechnen
durchschnittsalter = gewichtete_summe / gesamtbevölkerung

print(f"Der gewichtete Altersdurchschnitt im Landkreis Konstanz (2023) beträgt: {durchschnittsalter:.1f} Jahre")


Der gewichtete Altersdurchschnitt im Landkreis Konstanz (2023) beträgt: 42.4 Jahre


In [102]:
# ==========================================
# Aggregation der Gesamtbevölkerung 
# für den Landkreis Konstanz pro Jahr
# ==========================================

# Definition der Spalten, die aufsummiert werden sollen
spalten = ['insgesamt_dt', 'zusammen_aus', 'gesamtbevölkerung']

# Gruppierung nach Jahr und Summierung der relevanten Bevölkerungszahlen
# Ergebnis: für jedes Jahr die Gesamtzahl Deutscher, Ausländer
# sowie die Gesamtbevölkerung im Landkreis
df_landkreis_kn_sum = (
    df_landkreis_kn.groupby('jahr')[spalten]
    .sum()
    .reset_index()
)

# Beispielhafte Anzeige der aggregierten Zahlen
df_landkreis_kn_sum.head()


jahr  insgesamt_dt  zusammen_aus  gesamtbevölkerung
0  2014        273292         32909             306201
1  2015        277697         37110             314807
2  2016        279545         39254             318799
3  2017        281319         41395             322714
4  2018        282543         42844             325387

In [103]:
# ==========================================
# Pivotierung und Aufbereitung der Altersgruppen
# im Landkreis Konstanz
# ==========================================

# Pivot-Tabelle erstellen:
# Zeilen = Jahr
# Spalten = Altersgruppen
# Werte = Gesamtbevölkerung pro Altersgruppe
df_landkreis_kn_pivot = df_landkreis_kn.pivot_table(
    index='jahr',
    columns='alter_group',
    values='gesamtbevölkerung',
    aggfunc='sum'
).reset_index()

# Entfernen des Spaltennamens (optische Vereinfachung)
df_landkreis_kn_pivot.columns.name = None
df_landkreis_kn_pivot = df_landkreis_kn_pivot.rename_axis(None, axis=1)

print(df_landkreis_kn_pivot.head())

# Zusammenführen mit den aggregierten Gesamtzahlen je Jahr
# Ergebnis: ein DataFrame mit Gesamtzahlen + Altersgruppen-Spalten
df_landkreis_kn_gesamt = pd.merge(
    df_landkreis_kn_sum,
    df_landkreis_kn_pivot,
    on='jahr'
)

# Beispielhafte Anzeige
print(df_landkreis_kn_gesamt.head())

# ==========================================
# Berechnung von Prozentanteilen
# ==========================================

# Definition der Spalten, für die Prozentanteile gebildet werden sollen
cols_to_percent = [
    col for col in df_landkreis_kn.columns
    if col not in ['jahr', 'alter_group', 'gesamtbevölkerung']
]

# Neues DataFrame zur Aufnahme der Prozentanteile
df_landkreis_kn_prozent = pd.DataFrame()
df_landkreis_kn_prozent['jahr'] = df_landkreis_kn['jahr']
df_landkreis_kn_prozent['alter_group'] = df_landkreis_kn['alter_group']
df_landkreis_kn_prozent['gesamtbevölkerung'] = df_landkreis_kn['gesamtbevölkerung']

print(df_landkreis_kn_prozent.head())

# Prozentualer Anteil jeder Spalte an der Gesamtbevölkerung pro Altersgruppe
for col in cols_to_percent:
    df_landkreis_kn_prozent[col + '_prozent'] = (
        df_landkreis_kn[col] / df_landkreis_kn['gesamtbevölkerung'] * 100
    ).round(2)

# ==========================================
# Export
# ==========================================

# Gesamttabelle (absolute Zahlen + Altersgruppen) speichern
df_landkreis_kn_gesamt.to_csv(f"{output_dir}/df_landkreis_kn_gesamt.csv", index=False)

# Prozentanteile speichern
df_landkreis_kn_prozent.to_csv(f"{output_dir}/df_landkreis_kn_prozent.csv", index=False)


   jahr   0-17  18-29  30-39  40-49  50-59    60+
0  2014  49224  51786  37717  45897  45904  75673
1  2015  50663  55041  39437  45255  47553  76858
2  2016  51758  55536  40422  44067  48980  78036
3  2017  52066  56691  41533  42923  50150  79351
4  2018  52695  56515  42382  41907  51181  80707
   jahr  insgesamt_dt  zusammen_aus  gesamtbevölkerung   0-17  18-29  30-39  \
0  2014        273292         32909             306201  49224  51786  37717   
1  2015        277697         37110             314807  50663  55041  39437   
2  2016        279545         39254             318799  51758  55536  40422   
3  2017        281319         41395             322714  52066  56691  41533   
4  2018        282543         42844             325387  52695  56515  42382   

   40-49  50-59    60+  
0  45897  45904  75673  
1  45255  47553  76858  
2  44067  48980  78036  
3  42923  50150  79351  
4  41907  51181  80707  
   jahr alter_group  gesamtbevölkerung
0  2014        0-17              492

/var/folders/xb/3cn73kh15fx078zh0ryb_lrc0000gn/T/ipykernel_67769/1965284701.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  df_landkreis_kn_pivot = df_landkreis_kn.pivot_table(


In [104]:
print(df_landkreis_kn_prozent.tail())

    jahr alter_group  gesamtbevölkerung  insgesamt_dt_prozent  \
49  2023       18-29              54201                 82.69   
50  2023       30-39              46933                 78.72   
51  2023       40-49              41996                 80.09   
52  2023       50-59              48912                 85.69   
53  2023         60+              89973                 91.55   

    männlich_dt_prozent  weiblich_st_prozent  zusammen_aus_prozent  \
49                41.62                41.07                 17.31   
50                39.77                38.95                 21.28   
51                38.78                41.31                 19.91   
52                41.24                44.45                 14.31   
53                42.32                49.23                  8.45   

    männlich_aus_prozent  weiblich_aus_prozent  
49                  9.18                  8.13  
50                 11.12                 10.15  
51                  9.85                 

In [105]:
print(df_landkreis_kn_gesamt.tail())

   jahr  insgesamt_dt  zusammen_aus  gesamtbevölkerung   0-17  18-29  30-39  \
4  2018        282543         42844             325387  52695  56515  42382   
5  2019        283435         43922             327357  53519  56043  43296   
6  2020        283966         44596             328562  54153  54351  43964   
7  2021        285143         45868             331011  54877  54205  44628   
8  2023        288437         50454             338891  56876  54201  46933   

   40-49  50-59    60+  
4  41907  51181  80707  
5  40854  51378  82267  
6  40191  51718  84185  
7  40137  51085  86079  
8  41996  48912  89973  


# 10. Landkreis Konstanz: Demographie und Wahlergebnisse zusammenführen
In diesem Abschnitt werden die Wahlergebnisse für den Landkreis Konstanz mit den jeweils aktuellsten Demographie-Daten kombiniert. Die Wahlergebnisse stammen von der offiziellen Website der Bundeswahlleiterin: https://www.bundeswahlleiterin.de/bundestagswahlen/2013/ergebnisse/bund-99/land-8/wahlkreis-287.html (jeweils für jedes Jahr). 

- **Einlesen der Wahlergebnisse**: Die Wahldaten werden als DataFrame angelegt und enthalten die Stimmenanteile der wichtigsten Parteien für verschiedene Jahre.
- **Zuordnung der Demographie-Daten zu den Wahljahren**: Für jedes Wahljahr wird das jeweils aktuellste verfügbare Demographie-Jahr ausgewählt (ggf. mit Rückgriff auf das Vorjahr, falls für das Wahljahr selbst keine Daten vorliegen).
- **Kombination der Daten**: Die Demographie- und Wahldaten werden in einem DataFrame zusammengeführt.
- **Export des kombinierten DataFrames**: Das Ergebnis wird als CSV-Datei gespeichert und steht für weitere Analysen zur Verfügung.

In [106]:
# ==========================================
# Manuelles Erstellen eines DataFrames
# für die Zweitstimmenanteile im Landkreis Konstanz
# ==========================================

# Definition der Zweitstimmenanteile je Partei und Jahr
# (Quelle: Bundeswahlleiterin, Wahlkreis Konstanz)
# Hinweis: AfD wurde 2013 nicht explizit ausgewiesen,
# daher hier als 0% eingesetzt
data = {
    "Jahr": [2013, 2017, 2021, 2025],
    "CDU": [43.6, 33.2, 23.2, 30.1],
    "SPD": [20.2, 15.6, 20.9, 13.3],
    "GRÜNE": [12.7, 16.0, 21.0, 16.8],
    "FDP": [7.4, 13.2, 15.0, 5.8],
    "AfD": [0.0, 10.3, 8.5, 18.3],  # 2013 keine Angabe -> 0%
    "DIE LINKE": [5.0, 7.2, 3.7, 7.4],
    "Sonstige": [11.2, 4.6, 7.7, 8.2]
}

# Umwandlung in einen DataFrame
df_lk_kn_wahl = pd.DataFrame(data)

# Beispielhafte Ausgabe zur Kontrolle
print(df_lk_kn_wahl)


   Jahr   CDU   SPD  GRÜNE   FDP   AfD  DIE LINKE  Sonstige
0  2013  43.6  20.2   12.7   7.4   0.0        5.0      11.2
1  2017  33.2  15.6   16.0  13.2  10.3        7.2       4.6
2  2021  23.2  20.9   21.0  15.0   8.5        3.7       7.7
3  2025  30.1  13.3   16.8   5.8  18.3        7.4       8.2


In [107]:
# ==========================================
# Verknüpfung Demographie- und Wahldaten 
# für den Landkreis Konstanz
# ==========================================

# Ziel:
# Für jedes Wahljahr die passende demographische Struktur ergänzen.
# Falls für ein Wahljahr keine Demographie-Daten vorliegen,
# wird das nächste verfügbare Jahr herangezogen.
# Beispiel: Für 2013 gibt es keine Daten, also stattdessen 2014 nutzen.

rows = []

for wahljahr in df_lk_kn_wahl["Jahr"]:
    
    # Sonderfall: 2013
    # explizit auf 2014 mappen, falls 2013 nicht im Demographie-DataFrame vorhanden
    if wahljahr == 2013 and 2013 not in df_landkreis_kn_gesamt["jahr"].values:
        df_demo = df_landkreis_kn_gesamt[df_landkreis_kn_gesamt["jahr"] == 2014]
    
    # alle anderen Jahre: jeweils das aktuellste Jahr kleiner/gleich Wahljahr
    else:
        df_demo = df_landkreis_kn_gesamt[
            df_landkreis_kn_gesamt["jahr"] <= wahljahr
        ]
    
    # Prüfen, ob ein passender Datensatz gefunden wurde
    if not df_demo.empty:
        # immer den aktuellsten Eintrag nehmen
        demo_row = df_demo.sort_values("jahr", ascending=False).iloc[0]
        
        # passender Wahldatensatz für dieses Wahljahr
        wahl_row = df_lk_kn_wahl[df_lk_kn_wahl["Jahr"] == wahljahr].iloc[0]
        
        # kombiniere beide zu einem Dictionary
        merged = demo_row.to_dict()
        for col in df_lk_kn_wahl.columns:
            if col != "Jahr":
                merged[col] = wahl_row[col]
        
        # Wahl- und Demojahr dokumentieren
        merged["wahljahr"] = wahljahr
        merged["demo_jahr"] = demo_row["jahr"]
        
        rows.append(merged)

# ==========================================
# Endgültiges DataFrame
# ==========================================

# In ein DataFrame umwandeln
df_lk_kn_gesamt_wahl = pd.DataFrame(rows)

# Spaltenreihenfolge definieren
cols = ["wahljahr", "demo_jahr"] + [
    col for col in df_lk_kn_gesamt_wahl.columns
    if col not in ["wahljahr", "demo_jahr", "jahr"]
]
df_lk_kn_gesamt_wahl = df_lk_kn_gesamt_wahl[cols]

# Export als CSV
df_lk_kn_gesamt_wahl.to_csv(f"{output_dir}/df_lk_kn_gesamt_wahl.csv", index=False)

# Beispielausgabe
df_lk_kn_gesamt_wahl.head()


wahljahr  demo_jahr  insgesamt_dt  zusammen_aus  gesamtbevölkerung   0-17  \
0      2013       2014        273292         32909             306201  49224   
1      2017       2017        281319         41395             322714  52066   
2      2021       2021        285143         45868             331011  54877   
3      2025       2023        288437         50454             338891  56876   

   18-29  30-39  40-49  50-59    60+   CDU   SPD  GRÜNE   FDP   AfD  \
0  51786  37717  45897  45904  75673  43.6  20.2   12.7   7.4   0.0   
1  56691  41533  42923  50150  79351  33.2  15.6   16.0  13.2  10.3   
2  54205  44628  40137  51085  86079  23.2  20.9   21.0  15.0   8.5   
3  54201  46933  41996  48912  89973  30.1  13.3   16.8   5.8  18.3   

   DIE LINKE  Sonstige  
0        5.0      11.2  
1        7.2       4.6  
2        3.7       7.7  
3        7.4       8.2

# 11. Kanton Thurgau: Daten einlesen, bereinigen und aggregieren
In diesem Abschnitt werden demographische Daten für alle Bezirke des Kantons Thurgau aggregiert und bereinigt. Die Daten stammen vom Datenportal des Kantons Thurgau. 

- **Einlesen der Rohdaten**: Die Daten werden aus mehreren CSV- und Excel-Dateien geladen und enthalten Informationen zu Alter, Konfession, Staatsbürgerschaft und Bildung.
- **Bildung von Altersgruppen**: Die Altersdaten werden in vordefinierte Gruppen eingeteilt und pro Gemeinde und Bezirk aggregiert.
- **Aggregation von Konfessions- und Staatsbürgerschaftsdaten**: Die Daten werden auf Gemeinde- und Bezirksebene gruppiert und summiert.
- **Zusammenführung der Daten**: Die verschiedenen Datenquellen werden auf Gemeinde- und Bezirksebene zusammengeführt.
- **Einlesen und Bereinigung der Bildungsdaten**: Die Bildungsdaten werden aus einer Excel-Datei geladen, Spalten werden bereinigt und in numerische Werte umgewandelt (https://statistik.tg.ch/themen-und-daten/bildung-und-kultur/bildung/bildungsstand-der-bevoelkerung.html/6888#js-accordion_control--00)
- **Berechnung prozentualer Anteile der Bildungsniveaus**: Für jede Kategorie (hoch, mittel, niedrig) werden die prozentualen Anteile berechnet.
- **Export der aggregierten Daten**: Die Ergebnisse werden als CSV-Dateien gespeichert.

In [108]:
# ==========================================
# Aggregation aller Bezirke im Kanton Thurgau
# ==========================================

# Einlesen der Rohdaten
# Altersstruktur (sk-stat-57), Konfessionen (sk-stat-62),
# Staatsbürgerschaften und Geschlecht (sk-stat-59)
df57 = pd.read_csv(pfad + "sk-stat-57.csv")
df62 = pd.read_csv(pfad + "sk-stat-62.csv")
df59 = pd.read_csv(pfad + "sk-stat-59.csv")

# ==========================================
# Altersgruppen bilden für alle Bezirke
# ==========================================

# Definition der Altersgruppen
bins = [0, 17, 29, 39, 49, 59, 120]
labels = ["0-17", "18-29", "30-39", "40-49", "50-59", "60+"]

# Alterscodes in numerische Werte umwandeln
df57["ALTER"] = pd.to_numeric(df57["ALTER_CODE"], errors="coerce")

# Einteilung in Altersgruppen
df57["ALTERSGRUPPE"] = pd.cut(
    df57["ALTER"],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

# Aggregation der Altersgruppen pro Gemeinde
alter_agg = (
    df57.groupby(["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME", "ALTERSGRUPPE"])["ANZAHL_PERSONEN"]
    .sum()
    .reset_index()
    .pivot(
        index=["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME"],
        columns="ALTERSGRUPPE",
        values="ANZAHL_PERSONEN"
    )
    .reset_index()
)

# Gesamtbevölkerung pro Gemeinde berechnen
alter_agg["gesamtbevölkerung"] = alter_agg[labels].sum(axis=1)

# ==========================================
# Konfessionen pro Gemeinde aggregieren
# ==========================================

konf = (
    df62.groupby(["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME", "KONFESSION_BEZEICHNUNG"])["ANZAHL_PERSONEN"]
    .sum()
    .reset_index()
    .pivot(
        index=["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME"],
        columns="KONFESSION_BEZEICHNUNG",
        values="ANZAHL_PERSONEN"
    )
    .reset_index()
)

# ==========================================
# Staatsangehörigkeit und Geschlecht pro Gemeinde
# ==========================================

staatsb = (
    df59.groupby(["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME", "GESCHLECHT_BEZEICHNUNG", "NATIONALITAET_BEZEICHNUNG"])["ANZAHL_PERSONEN"]
    .sum()
    .reset_index()
    .pivot(
        index=["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME"],
        columns=["GESCHLECHT_BEZEICHNUNG", "NATIONALITAET_BEZEICHNUNG"],
        values="ANZAHL_PERSONEN"
    )
    .reset_index()
)

# Spaltennamen aus den MultiIndex-Spalten glätten
staatsb.columns = [
    "_".join([str(i) for i in col if i]) if isinstance(col, tuple) else col
    for col in staatsb.columns.values
]

# ==========================================
# Merge aller demographischen Tabellen
# auf Gemeinde-Ebene
# ==========================================

result_gemeinde = (
    alter_agg
    .merge(konf, on=["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME"], how="left")
    .merge(staatsb, on=["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME"], how="left")
)

# ==========================================
# NEU: Aggregation auf Bezirksebene
# ==========================================

# Alle numerischen Spalten, außer den Indexspalten, summieren
index_cols = ["JAHR", "BEZIRK_NAME"]
num_cols = [
    col for col in result_gemeinde.columns
    if col not in index_cols + ["GEMEINDE_NAME"]
]

# Summierung auf Bezirksebene über alle Gemeinden hinweg
result_bezirk = (
    result_gemeinde.groupby(index_cols)[num_cols]
    .sum(min_count=1)
    .reset_index()
)

# Beispielhafte Anzeige
result_bezirk.head()


/var/folders/xb/3cn73kh15fx078zh0ryb_lrc0000gn/T/ipykernel_67769/1505980917.py:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df57.groupby(["JAHR", "BEZIRK_NAME", "GEMEINDE_NAME", "ALTERSGRUPPE"])["ANZAHL_PERSONEN"]


JAHR         BEZIRK_NAME   0-17  18-29  30-39  40-49  50-59    60+  \
0  2015        Bezirk Arbon   9812   8441   7073   7820   8489  13065   
1  2015   Bezirk Frauenfeld  11832   9782   8585   9482  10511  15216   
2  2015  Bezirk Kreuzlingen   7918   6617   6396   7118   7553  10894   
3  2015   Bezirk Münchwilen   8839   7136   6009   6928   7197   9753   
4  2015   Bezirk Weinfelden  10271   8964   7089   7821   8301  11598   

   gesamtbevölkerung  Evangelisch-reformiert  Römisch-katholisch   Übrige  \
0              54700                 16747.0             18165.0  19788.0   
1              65408                 26672.0             19691.0  19045.0   
2              46496                 15574.0             13698.0  17224.0   
3              45862                 13951.0             18455.0  13456.0   
4              54044                 21219.0             16328.0  16497.0   

   Männlich_Ausland  Männlich_Schweiz  Weiblich_Ausland  Weiblich_Schweiz  
0            7584.0           19621.0            6630.0           20865.0  
1            6935.0           25717.0            5819.0           26937.0  
2            9673.0           13562.0            8723.0           14538.0  
3            4432.0           18916.0            3620.0           18894.0  
4            5786.0           21682.0            4682.0           21894.0

In [113]:
# ==========================================
# Einlesen und Aufbereitung der Bildungsdaten 
# für den Kanton Thurgau (Stand 2024)
# ==========================================

# Dateipfad zur Excel-Datei mit der höchsten abgeschlossenen Ausbildung
# nach Gemeinden und Bezirken
file_path = "/Users/tessabartels/Desktop/Coding/Datenjournalismus/data/Kreuzlingen:Kanton Thurgau/T3_2025_Hoechste abgeschlossene Ausbildung der Bevoelkerung nach GemeindenBezirke_ab_2010.xlsx"

# Einlesen der relevanten Spalten aus der Excel-Datei
# - skiprows=6: Überspringt Kopfzeilen
# - skipfooter=12: Entfernt Fußnoten am Ende
# - usecols: Auswahl nur der benötigten Spalten
df_bildung_ktg = pd.read_excel(
    file_path,
    skiprows=6,
    skipfooter=12,
    usecols=[0, 1, 5, 9, 13],
    engine="openpyxl"
)

# Umbenennen der Spalten für bessere Lesbarkeit
df_bildung_ktg.columns = ["Gebiet", "gesamt", "niedrig", "mittel", "hoch"]

# Bereinigung der Zahlenwerte:
# - Entfernt Sternchen oder Leerzeichen
# - Wandelt Werte in Integer (mit fehlenden Werten als NA)
for col in ["gesamt", "niedrig", "mittel", "hoch"]:
    df_bildung_ktg[col] = (
        df_bildung_ktg[col]
        .astype(str)
        .str.replace("*", "", regex=False)
        .str.replace(" ", "")
        .replace("", pd.NA)
        .astype("Int64")
    )

# ==========================================
# Prozentuale Bildungsanteile berechnen
# ==========================================

df_bildung_ktg["anteil_niedrig"] = (
    df_bildung_ktg["niedrig"] / df_bildung_ktg["gesamt"] * 100
).round(1)

df_bildung_ktg["anteil_mittel"] = (
    df_bildung_ktg["mittel"] / df_bildung_ktg["gesamt"] * 100
).round(1)

df_bildung_ktg["anteil_hoch"] = (
    df_bildung_ktg["hoch"] / df_bildung_ktg["gesamt"] * 100
).round(1)

# ==========================================
# Export als CSV
# ==========================================

df_bildung_ktg.to_csv(f"{output_dir}/df_bildung_ktg.csv", index=False)

# Beispielhafte Anzeige
df_bildung_ktg.head()


Gebiet  gesamt  niedrig  mittel   hoch  anteil_niedrig  \
0  Bezirk Arbon   31263     5154   14953  11156            16.5   
1      Amriswil    7669     1622    3736   2310            21.2   
2         Arbon    7655     1388    3563   2703            18.1   
3        Dozwil     371       35     205    131             9.4   
4        Egnach    2685      263    1306   1115             9.8   

   anteil_mittel  anteil_hoch  
0           47.8         35.7  
1           48.7         30.1  
2           46.5         35.3  
3           55.3         35.3  
4           48.6         41.5

## Für Kreuzlingen auch erstellen: 

In [114]:
# Liste aller relevanten Gebiete im Bezirk Kreuzlingen
gebiete_kr = [
    "Bezirk Kreuzlingen",
    "Altnau", "Bottighofen", "Ermatingen", "Gottlieben", "Güttingen", 
    "Kemmental", "Kreuzlingen", "Langrickenbach", "Lengwil", 
    "Münsterlingen", "Raperswilen", "Salenstein", "Tägerwilen", "Wäldi"
]

# Filter
df_bildung_ktg_kr = df_bildung_ktg[df_bildung_ktg["Gebiet"].isin(gebiete_kr)].reset_index(drop=True)


# Speichern
df_bildung_ktg_kr.to_csv(f"{output_dir}/df_bildung_ktg_kr.csv", index=False)
df_bildung_ktg_kr

Gebiet  gesamt  niedrig  mittel   hoch  anteil_niedrig  \
0   Bezirk Kreuzlingen   28681     3207   11375  14099            11.2   
1               Altnau    1269       66     541    663             5.2   
2          Bottighofen    1578      126     432   1020             8.0   
3           Ermatingen    1944      130     836    978             6.7   
4           Gottlieben     257     <NA>     100    135            <NA>   
5            Güttingen     911       83     351    477             9.1   
6            Kemmental    1560      143     887    530             9.2   
7          Kreuzlingen   12730     1873    4546   6311            14.7   
8       Langrickenbach     669       61     296    313             9.1   
9              Lengwil    1093      118     511    463            10.8   
10       Münsterlingen    1980      216     854    909            10.9   
11         Raperswilen     218     <NA>     133     62            <NA>   
12          Salenstein     916       59     345    513             6.4   
13          Tägerwilen    2897      212    1192   1494             7.3   
14               Wäldi     659       76     353    230            11.5   

    anteil_mittel  anteil_hoch  
0            39.7         49.2  
1            42.6         52.2  
2            27.4         64.6  
3            43.0         50.3  
4            38.9         52.5  
5            38.5         52.4  
6            56.9         34.0  
7            35.7         49.6  
8            44.2         46.8  
9            46.8         42.4  
10           43.1         45.9  
11           61.0         28.4  
12           37.7         56.0  
13           41.1         51.6  
14           53.6         34.9


# 12. Kanton Thurgau: Demographie und Wahlergebnisse zusammenführen

In diesem Abschnitt werden die demographischen Daten mit den Wahlergebnissen für den Kanton Thurgau zusammengeführt. Die Wahldaten stammen vom Amt für Daten und Statistik Thurgau: https://statistik.tg.ch/themen-und-daten/staat-und-politik/wahlen-und-abstimmungen/nationalratswahlen.html/10125#js-accordion_control--00 
- **Einlesen der Parteistärken**: Die Wahlergebnisse (Parteistärken) werden aus Excel-Dateien für verschiedene Jahre eingelesen (siehe oben)
- **Bereinigung und Vereinheitlichung der Spalten**: Die Spaltennamen werden je nach Jahr angepasst und die Werte in numerische Formate konvertiert.
- **Zusammenführen mit den demographischen Daten auf Bezirksebene**: Die Parteistärken werden mit den aggregierten demographischen Daten auf Bezirksebene gemerged.
- **Export der kombinierten Daten**: Die zusammengeführten Daten werden als CSV-Datei gespeichert und stehen für weitere Analysen zur Verfügung.


In [111]:
# ==========================================
# Verknüpfung von Parteistärken und
# demographischen Daten auf Bezirksebene
# ==========================================

# Dictionary mit den Parteistärken pro Jahr
# bereits eingelesen und aufbereitet
parteistaerke_dict = {
    2015: kt_parteistaerke_2015,
    2019: kt_parteistaerke_2019,
    2023: kt_parteistaerke_2023
}

# Liste zur Aufnahme der Ergebnisse pro Jahr
merged_bezirk_results = []

# Iteration über alle Jahre und deren Parteistärken
for jahr, partei_df in parteistaerke_dict.items():
    
    
    
    # Filterung des result_bezirk-DataFrames auf das jeweilige Jahr
    bezirk_jahr = result_bezirk[result_bezirk["JAHR"] == jahr].copy()
    
    # Merge über den Bezirk-Namen
    merged = bezirk_jahr.merge(
        partei_df,
        left_on="BEZIRK_NAME",
        right_on="Gebiet",
        how="left",   # alle Bezirke bleiben enthalten
        suffixes=('', f'_parteistaerke_{jahr}')
    )
    
    # Jahr der Wahl explizit speichern
    merged["WAHLJAHR"] = jahr
    
    # Ergebnis zur Liste hinzufügen
    merged_bezirk_results.append(merged)

# ==========================================
# Alle Jahre in einen gemeinsamen DataFrame zusammenführen
# ==========================================
df_bezirk_parteistaerke = pd.concat(merged_bezirk_results, ignore_index=True)



# ==========================================
# Export als CSV
# ==========================================

df_bezirk_parteistaerke.to_csv(f"{output_dir}/df_bezirk_parteistaerke_kt.csv", index=False)

# Beispielhafte Anzeige
df_bezirk_parteistaerke.head()


JAHR         BEZIRK_NAME   0-17  18-29  30-39  40-49  50-59    60+  \
0  2015        Bezirk Arbon   9812   8441   7073   7820   8489  13065   
1  2015   Bezirk Frauenfeld  11832   9782   8585   9482  10511  15216   
2  2015  Bezirk Kreuzlingen   7918   6617   6396   7118   7553  10894   
3  2015   Bezirk Münchwilen   8839   7136   6009   6928   7197   9753   
4  2015   Bezirk Weinfelden  10271   8964   7089   7821   8301  11598   

   gesamtbevölkerung  Evangelisch-reformiert  ...  BDP  EDU  EVP  DPS  \
0              54700                 16747.0  ...  3.4  3.9  2.4  0.2   
1              65408                 26672.0  ...  4.0  2.4  2.6  0.2   
2              46496                 15574.0  ...  3.6  2.1  1.5  0.1   
3              45862                 13951.0  ...  3.5  2.6  2.1  0.1   
4              54044                 21219.0  ...  4.2  5.8  2.8  0.2   

   WAHLJAHR  Die Mitte GLP  GRÜNE  AUFTG  Mass-Voll  
0      2015        NaN NaN    NaN    NaN        NaN  
1      2015        NaN NaN    NaN    NaN        NaN  
2      2015        NaN NaN    NaN    NaN        NaN  
3      2015        NaN NaN    NaN    NaN        NaN  
4      2015        NaN NaN    NaN    NaN        NaN  

[5 rows x 33 columns]